# Data Preparation

In [ ]:
# Change directory to keep paths consistent
%cd /Users/brk/projects/masters/SU/ergo/src

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib as mpl
# # NOTE: These two lines let us preview the plots in a notebook, but the inline
# # plots will have the *wrong font* ):
# from matplotlib.backends.backend_pgf import FigureCanvasPgf
# mpl.backend_bases.register_backend('pdf', FigureCanvasPgf)
# import pandas as pd
# import numpy as np

# plt.rcParams.update({
#     'font.family': 'Cambria',
#     'text.usetex': True,
#     "pgf.rcfonts": False,
#     'pgf.texsystem': 'xelatex',
#     'pgf.preamble': '\n'.join([
#         r'\usepackage[math-style=TeX,bold-style=TeX]{unicode-math}',
#         r'\setmainfont{Cambria}',
#         r'\setmathfont{Cambria Math}',
#     ])
# })

# y_trn = np.random.randint(0, 50, (1000,))
# WIDTH = 5.5
# fig, axs = plt.subplots(1, 2, figsize=(WIDTH, WIDTH*.5), dpi=200)
# counts = pd.Series(y_trn).value_counts()
# axs[0].bar(counts.index, counts)

# counts = pd.Series(y_trn[y_trn != 50]).value_counts()
# axs[1].bar(counts.index, counts)

# axs[0].set(
#     title='Number of classes\n',
#     xlabel='Class',
#     ylabel='Count',
# )
# axs[1].set_title(
#     'Number of classes\n(excluding class 12345)$12345\\frac{1}{2}e^{-x}$',
# )
# axs[1].set(
# #     title=',
#     xlabel='Class',
#     ylabel='Count',
# )

# plt.tight_layout()
# plt.savefig('tmp.pdf')
# plt.show()

## Imports and setup

In [ ]:
# minimise me
%load_ext autoreload
%autoreload 2
import seaborn as sns
import seaborn.objects as so
# import matplotlib.pyplot as plt
# import matplotlib as mpl
import ipywidgets as widgets
import datetime
from ipywidgets import interact, interactive, fixed, interact_manual
import pandas as pd
import numpy as np
import models
import vis
import common
import read
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
import sklearn
import tqdm
import logging as l
import tqdm
import yaml
import glob
from matplotlib.colors import LogNorm
import re
from sklearn.metrics import classification_report
from scipy.stats import f

import matplotlib.pyplot as plt
import matplotlib as mpl
# NOTE: These two lines let us preview the plots in a notebook, but the inline
# plots will have the *wrong font* ):
from matplotlib.backends.backend_pgf import FigureCanvasPgf
mpl.backend_bases.register_backend('pdf', FigureCanvasPgf)
import pandas as pd
import numpy as np

plt.rcParams.update({
    'font.family': 'Cambria',
    'text.usetex': True,
    "pgf.rcfonts": False,
    'pgf.texsystem': 'xelatex',
    'pgf.preamble': '\n'.join([
        r'\usepackage[math-style=TeX,bold-style=TeX]{unicode-math}',
        r'\setmainfont{Cambria}',
        r'\setmathfont{Cambria Math}',
    ])
})

# plt.rcParams.update({
#     'font.size' : 9,
#     'axes.labelsize': 9,
#     'legend.fontsize': 9,
#     'font.family': 'Cambria',
#     "pgf.rcfonts": False,
#     'pgf.texsystem': 'xelatex',
#     'pgf.preamble': '\n'.join(['',
#         r'\usepackage[math-style=TeX,bold-style=TeX]{unicode-math}',
#         r'\setmainfont{Cambria}',
#         r'\setmathfont{Cambria Math}',
#     ])
# })

# plt.rcParams.update({
#     'font.size' : 9,                   # Set font size to 11pt
#     'axes.labelsize': 9,               # -> axis labels
#     'legend.fontsize': 9,              # -> legends
#     'font.family': 'serif',
#     'text.usetex': True,
#     'text.latex.preamble': (            # LaTeX preamble
#         r'\usepackage{lmodern}'
#     )
# })

## Utility functions

In [ ]:
# minimise me
def prettify_col_name(x):
    return x.split('.')[-1].replace('_', ' ').title()

def calculate_prediction_ellipse(x, y, alpha=0.95):
    """Given some x and y data, calculate the (1-alpha) confidence ellipse."""
    data = np.column_stack((x, y)) # Combine x and y into a single data array
    num_dimensions = data.shape[1]
    num_data_points = data.shape[0]
    # Estimate the sample covariance matrix
    sample_covariance_matrix = np.cov(data, rowvar=False)
    # Calculate the sample mean for each dimension
    sample_mean = np.mean(data, axis=0)
    # Generate angles for the ellipse
    theta = np.linspace(0, 2*np.pi, num=100)
    # Calculate the radius of the ellipse. `f.ppf` is the inverse of the CDF
    radius = np.sqrt(
        num_dimensions * (num_data_points - 1) / (num_data_points - num_dimensions) *
        (1 + 1/num_data_points) * f.ppf(1 - alpha, num_dimensions, num_data_points - num_dimensions)
    )
#     print(sample_covariance_matrix)
    # Compute the Cholesky decomposition of the covariance matrix
    chol_cov_matrix = np.linalg.cholesky(sample_covariance_matrix)
    # Generate ellipse offset based on Cholesky decomposition
    ellipse_offset = np.outer(np.cos(theta), chol_cov_matrix[0, :]) + np.outer(np.sin(theta), chol_cov_matrix[1, :])
    # Calculate the points of the prediction interval ellipse
    prediction_ellipse_points = sample_mean + radius * ellipse_offset
    return prediction_ellipse_points

def get_npz_data_from_model(model_dir):
    """Given a directory of a model, return it's y_pred and y_true."""
    data = np.load(f'{model_dir}/y_val_true_y_val_pred.npz')
    y_true = data['y_true']
    y_pred = data['y_pred']
    return y_true, y_pred

def show_conf_mat_from_model(model_dir, ax=None):
    """Given a directory of a model, plot its confidence matrix"""
    y_true, y_pred = get_npz_data_from_model(model_dir)
    cm_val = tf.math.confusion_matrix(
        y_true.flatten(), 
        y_pred.flatten()
    ).numpy()
    p = vis.conf_mat(cm_val / cm_val.sum(axis=0), ax=ax)
    return p

## Load data

In [ ]:
# Read in data from hpar optimisation
paths = sorted(glob.glob('../saved_models/results_*_optuna.jsonl'))
print(f'Reading data from\n', "\n".join(paths))
dfs = map(
    lambda path: pd.read_json(path, lines=True),
    paths
)
# Concat the dataframes together, and then do a 
# copy to avoid a dataframe fragmentation warning
# Reset the index to avoid a seaborn error https://github.com/mwaskom/seaborn/issues/3291
df = pd.concat(dfs).reset_index(drop=True).copy()
df['preprocessing.num_gesture_classes'] = df['preprocessing.num_gesture_classes'].fillna('51')
df['preprocessing.num_gesture_classes'] = df['preprocessing.num_gesture_classes'].astype(int).astype(str)

# 50-class HFFNNs don't make sense, remove them
df = df[~(
    (df['model_type'] == 'HFFNN')
    & (df['preprocessing.num_gesture_classes'] == '50')
)]

df.groupby(['model_type', 'preprocessing.num_gesture_classes']).size()

## Constants

In [ ]:
model_colours = {
    'FFNN': 'tab:blue',
    'HFFNN': 'tab:orange',
    'CuSUM': 'tab:green',
    'HMM': 'tab:red',
    'SVM': 'tab:purple',
}
palette = 'Spectral'
other_colours = [
    'tab:brown',
    'tab:pink',
    'tab:grey',
    'tab:olive',
    'tab:cyan',
]
WIDTH = 5.5
BIG_WIDTH = WIDTH * 1.4

rename_hpars = {
    'ffnn.dropout_rate'            : 'Dropout Rate',
    'ffnn.l2_coefficient.log10'    : 'L2 ($\log_{10}$)',
    'nn.batch_size.log10'          : 'Batch Size ($\log_{10}$)',
    'nn.learning_rate.log10'       : 'LR ($\log_{10}$)',
    'ffnn.l2_coefficient'          : 'L2 Coef.',
    'nn.batch_size'                : 'Batch Size',
    'nn.learning_rate'             : 'LR',
    'ffnn.num_layers'              : '\#Layers',
    'ffnn.nodes_per_layer.1'       : '\#Nodes (layer 1)',
    'ffnn.nodes_per_layer.2'       : '\#Nodes (layer 2)',
    'ffnn.nodes_per_layer.3'       : '\#Nodes (layer 3)',
    'ffnn.nodes_per_layer.-1'      : '\#Nodes (last layer)',
    'ffnn.nodes_per_layer.1.log10' : '\#Nodes (layer 1, $\log_{10}$)',
    'ffnn.nodes_per_layer.2.log10' : '\#Nodes (layer 2, $\log_{10}$)',
    'ffnn.nodes_per_layer.3.log10' : '\#Nodes (layer 3, $\log_{10}$)',
    'ffnn.nodes_per_layer.-1.log10': '\#Nodes (last layer, $\log_{10}$)',
    'val.macro avg.f1-score'       : '$F_1$-score',
    'val.macro avg.recall'         : 'Recall',
    'val.macro avg.precision'      : 'Precision',
    'val.loss.log10'               : 'Val. Loss ($\log_{10}$)',
    'trn.loss.log10'               : 'Trn. Loss ($\log_{10}$)',
    'val.loss'                     : 'Val. Loss',
    'trn.loss'                     : 'Trn. Loss',
}
# Add rename cols for the HFFNNs
rename_hpars |= {
    f'hffnn.majority.{k}': f'Maj. {v}' for k, v in rename_hpars.items() if 'nn.' in k
} | {
    f'hffnn.minority.{k}': f'Min. {v}' for k, v in rename_hpars.items() if 'nn.' in k
}

stripplot_kwargs = dict(
    s=2.5,
    linewidth=.1,
    alpha=0.75,
    edgecolor='#000a',
)

scatterplot_kwargs = dict(
    s=5,
    edgecolor='#000a',
    linewidth=0.1,
    alpha=0.75,
)

## Calculate some auxillary values

In [ ]:
# Preprocess the data a little bit, and get a list of dependant variables
# Preprocess the df a bit to get some nice-to-use columns

# Convert nodes_per_layer into nodes_per_layer.1, .2, .3
prefixes = (
    'ffnn.nodes_per_layer',
    'hffnn.majority.ffnn.nodes_per_layer',
    'hffnn.minority.ffnn.nodes_per_layer',
)
for prefix in prefixes:
    for i in (1, 2, 3):
        df[f'{prefix}.{i}'] = df[prefix].apply(
            lambda x: x[i-1] if isinstance(x, list) and len(x) >= i else None
        )
    df[f'{prefix}.-1'] = df[prefix].apply(
        lambda x: x[-1] if isinstance(x, list) and len(x) >= 1 else None
    )


# Calculate ratios
avgs = ('macro avg', 'weighted avg')
metrics = ('f1-score', 'precision', 'recall')

for avg in avgs:
    for metric in metrics:
        df[f'ratio.{avg}.{metric}'] = df[f'trn.{avg}.{metric}'] / df[f'val.{avg}.{metric}']
        df[f'ratio.{avg}.{metric}'] = np.where(
            np.isfinite(df[f'ratio.{avg}.{metric}']),
            df[f'ratio.{avg}.{metric}'],
            np.nan
        )
df['ratio.loss'] = df['trn.loss'] / df['val.loss']

# Print out a list of dependant variables
# dep_vars = sorted([
#     c for c in df.columns 
#     if 'val' not in c and 'trn' not in c and 'ratio' not in c and c not in (
#         'saved_at', 'fit_time', 'preprocessing.gesture_allowlist', 
# )], key=lambda c: str(c))
# print(f"Dependant variables: {dep_vars}")
# print("\nVariables which change:")
# max_len = max(map(lambda x: len(x), dep_vars))
# Print out all dependant variables that change
# for var in dep_vars:
#     uniq = df[var].apply(lambda x: str(x) if isinstance(x, list) else x).unique()
#     if len(uniq) > 1:
#         print(f"{var: <{max_len}} {uniq}")
        
df['ffnn.dropout_rate'] = np.round(df['ffnn.dropout_rate'], 6)

df['val.pred_time_per_obs'] = df['val.pred_time'] / df['val.num_observations']
df['trn.pred_time_per_obs'] = df['trn.pred_time'] / df['trn.num_observations']
df['fit_time_per_obs'] = df['fit_time'] / df['trn.num_observations']


# Add some log10 columns
log10_cols = [
    'val.loss',
    'trn.loss',
    'ffnn.l2_coefficient',
    'nn.batch_size',
    'nn.learning_rate',
    'ffnn.nodes_per_layer.1',
    'ffnn.nodes_per_layer.2',
    'ffnn.nodes_per_layer.3',
    'ffnn.nodes_per_layer.-1',
    'hffnn.majority.ffnn.nodes_per_layer.1',
    'hffnn.minority.ffnn.nodes_per_layer.1',
    'hffnn.majority.ffnn.nodes_per_layer.2',
    'hffnn.minority.ffnn.nodes_per_layer.2',
    'hffnn.majority.ffnn.nodes_per_layer.3',
    'hffnn.minority.ffnn.nodes_per_layer.3',
    'hffnn.majority.ffnn.nodes_per_layer.-1',
    'hffnn.minority.ffnn.nodes_per_layer.-1',
    'hffnn.majority.ffnn.l2_coefficient',
    'hffnn.minority.ffnn.l2_coefficient',
    'hffnn.majority.nn.batch_size',
    'hffnn.minority.nn.batch_size',
    'hffnn.majority.nn.learning_rate',
    'hffnn.minority.nn.learning_rate',
]

df[[f'{c}.log10' for c in log10_cols]] = np.log10(df[log10_cols])
# df[[f'{c}.log10' for c in log10_cols]] = df[[f'{c}.log10' for c in log10_cols]].fillna(0)

for c in log10_cols:
    if df[f'{c}.log10'].isna().any():
        print(f"{df[f'{c}.log10'].isna().sum()} NaNs in {c}.log10")

# There are a *lot* of columns. Here's a more-useful subset
subset_cols = [
    c for c in df.columns
    if (not re.search(r'((trn|val)\.\d+\.)|weighted avg', c)) and 
        (c not in [
            'hmm', 'lstm', 'ffnn', 'nn', 'hffnn', 'cusum', 'svm',
            'preprocessing.n_timesteps',
            'preprocessing.gesture_allowlist',
            'preprocessing.gesture_allowlist',
        ])
]

df['ffnn.num_layers'] = df['ffnn.nodes_per_layer'].apply(
    lambda x: np.nan if type(x) is not list else len(x)
)
df['hffnn.majority.ffnn.num_layers'] = df['hffnn.majority.ffnn.nodes_per_layer'].apply(
    lambda x: np.nan if type(x) is not list else len(x)
)
df['hffnn.minority.ffnn.num_layers'] = df['hffnn.minority.ffnn.nodes_per_layer'].apply(
    lambda x: np.nan if type(x) is not list else len(x)
)

In [ ]:
df[
    (df['model_type'] == "HFFNN")
    & (df['preprocessing.num_gesture_classes'] == '51')
].groupby(['hffnn.majority.ffnn.num_layers']).size()

In [ ]:
print("Performance charactersitcs of the models")
(
    df
    .groupby(['model_type', 'preprocessing.num_gesture_classes'])
    ['val.macro avg.f1-score']
    .agg(['min', 'median', 'max'])
#     .round(3)
)

In [ ]:
%%script false --no-raise-error
fig, axs = plt.subplots(3, 1, figsize=(WIDTH, WIDTH))
ngestures = ('51', '50', '5')
xmin = None
xmax = None
for ax, ngesture in zip(axs, ngestures):
    sns.scatterplot(
        data=df[df['preprocessing.num_gesture_classes'] == ngesture],
        x='saved_at',
        y='preprocessing.seed',
        hue='model_type',
        s=10,
        hue_order=list(model_colours.keys()),
    #     alpha=0.1,
        ax=ax,
    edgecolor=None,
    )
    ax.set_title(f'{ngesture} gestures')
#     if xmin is None: xmin = ax.get_xlim()[0]
#     if xmax is None: xmax = ax.get_xlim()[1]
#     xmin = min(xmin, ax.get_xlim()[0])
#     xmax = min(xmax, ax.get_xlim()[1])
# for ax in axs:
#     ax.set_xlim((xmin, xmax))
plt.tight_layout()

print(df.shape)
df = df[
    (df['model_type'] != 'HFFNN')
    | (df['saved_at'] > pd.to_datetime('2023-10-01T11:00:00'))
]
print(df.shape)

# Plotting

## Bar plot of number of observations

In [ ]:
%%script false --no-raise-error # This cell is unused
(
    X_trn, X_val, y_trn, y_val, dt_trn, dt_val
) = common.read_and_split_from_npz("../gesture_data/trn_20_10.npz")

fig, axs = plt.subplots(1, 2, figsize=(WIDTH, WIDTH*.5))
counts = pd.Series(y_trn).value_counts()
axs[0].bar(counts.index, counts)


counts = pd.Series(y_trn[y_trn != 50]).value_counts()
axs[1].bar(counts.index, counts)

axs[0].set(
    title='Number of classes\n',
    xlabel='Class',
    ylabel='Count',
)
axs[1].set(
    title='Number of classes\n(excluding class 50)',
    xlabel='Class',
    ylabel='Count',
)
plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_class_imbalance.pdf',
    bbox_inches='tight'
)
plt.show()

### 51 classes, Precision vs Recall vs $F_1$

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(WIDTH, WIDTH*0.5))

# recall_grid, precision_grid = np.meshgrid(
#     np.linspace(0, 1, 100), 
#     np.linspace(0, 1, 100)
# )
# f1_score = 2 * (precision_grid * recall_grid) / (precision_grid + recall_grid)

# contours = axs[0].contour(
#     recall_grid, 
#     precision_grid,
#     f1_score, 
#     levels=np.linspace(0.1, 1, 10), 
#     colors='black',
#     alpha=0.25
# )
# axs[0].clabel(contours, inline=True, fontsize=8, fmt='%.2f')

sns.scatterplot(
    data=df[df['preprocessing.num_gesture_classes'] == '51'],
    x='val.macro avg.precision',
    y='val.macro avg.recall',
#     s=5,
#     alpha=0.5,
    hue='model_type',
    hue_order=list(model_colours.keys()),
    ax=axs[1],
#     edgecolor=None,
    **scatterplot_kwargs,
    legend=False,
)
axs[1].set_xlim((-0.1, 1.1))
axs[1].set_ylim((-0.1, 1.1))
# axs[1].plot([0,1], [0,1], color='black', alpha=.1)
axs[1].set_title(f'Precision vs Recall\n51-classes')
axs[1].set_xlabel(f'Precision')
axs[1].set_ylabel(f'Recall')
# axs[0].legend().set_title("Model ")


order = ['FFNN', 'SVM', 'HFFNN', 'HMM', 'CuSUM']
sns.stripplot(
    data=df[df['preprocessing.num_gesture_classes'] == '51'],
    y='val.macro avg.f1-score',
    x='model_type',
#     s=2,
#     alpha=0.5,
#     order=list(model_colours.keys()),
    **stripplot_kwargs,
    jitter=0.3, 
    order=order,
    hue='model_type',
    hue_order=list(model_colours.keys()),
#     hue_order=order,
    ax=axs[0],
    legend=False,
)
axs[0].set_title(f'$F_1$-score vs Model Type\n51-classes')
axs[0].set_xlabel(f'Model Type')
axs[0].set_ylabel(f'$F_1$-score')
axs[0].set_ylim((-0.1, 1.1))
axs[0].grid(axis='y')

vis.add_grid(axs[0])
vis.add_grid(axs[1])
plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_precision_recall_51_classes.pdf', 
    bbox_inches='tight'
)


In [ ]:
%%script false --no-raise-error # This cell is unused
fig, ax = plt.subplots(1, 1, figsize=(WIDTH, WIDTH))

data = df.loc[
    df['preprocessing.num_gesture_classes'] == '51',
    ['model_type', 'val.macro avg.recall', 'val.macro avg.precision']
].melt(
    id_vars=['model_type'], 
    var_name='metric', 
    value_name='value'
)
data['metric'] = data['metric'].replace({
    'val.macro avg.recall': 'Recall',
    'val.macro avg.precision': 'Precision',
})

sns.stripplot(
    data=data,
    x="model_type", 
    y="value", 
    hue="metric",
    order=list(model_colours.keys()),
    palette=palette,
    dodge=True,
    alpha=0.75,
    size=2,
    ax=ax
)
ax.set_ylim((-0.05, 1.05))

ax.set_title(f'Precision and recall for all model types')
ax.set_xlabel(f'Model Type')
ax.set_ylabel(f'Metric Value')
ax.legend().set_title("Metric")
ax.set_yticks(np.arange(0, 1.1, .1))
ax.set_yticklabels(np.round(np.arange(0, 1.1, .1), 1))
ax.grid(axis='y')

plt.savefig(
    '../../report/src/imgs/graphs/05_precision_recall_stripplot.pdf', 
    bbox_inches='tight'
)
plt.tight_layout()

### Precision vs Recall for all models

In [ ]:
%%script false --no-raise-error # This cell is unused

# Three plots showing the precision and recall for all models.
# Each plot showing either 51, 50, or 5 gesture classes
fig, axs = plt.subplots(1, 3, figsize=(WIDTH, WIDTH/3.))
n_gesture_classes = ('51', '50', '5')

# recall_grid, precision_grid = np.meshgrid(
#     np.linspace(0, 1, 100), 
#     np.linspace(0, 1, 100)
# )
# f1_score = 2 * (precision_grid * recall_grid) / (precision_grid + recall_grid)

for ax, n_classes in zip(axs, n_gesture_classes):
#     contours = ax.contour(
#         recall_grid, 
#         precision_grid,
#         f1_score, 
#         levels=np.linspace(0.1, 1, 5), 
#         colors='black',
#         alpha=0.25
#     )
#     ax.clabel(contours, inline=True, fontsize=8, fmt='%.2f')
    
    sns.scatterplot(
        data=df[df['preprocessing.num_gesture_classes'] == n_classes],
        x='val.macro avg.precision',
        y='val.macro avg.recall',
        s=2.5,
        alpha=0.5,
        hue='model_type',
        hue_order=list(model_colours.keys()),
        ax=ax,
        legend=False,
        edgecolor=None,
    )
    ax.set_xlim((-0.1, 1.1))
    ax.set_ylim((-0.1, 1.1))
    ax.set_xlabel('Precision')
    ax.set_ylabel('Recall')
    ax.plot([0,1], [0,1], color='black', alpha=.1)
    ax.set_title(f'{n_classes} gesture classes')
#     ax.legend().set_title('Model Type')

plt.tight_layout()

In [ ]:
%%script false --no-raise-error # This cell is unused
# Three plots showing the precision and recall for all models.
# Each plot showing either 51, 50, or 5 gesture classes
# n_gesture_classes = ('51', '50', '5')

# recall_grid, precision_grid = np.meshgrid(
#     np.linspace(0, 1, 100), 
#     np.linspace(0, 1, 100)
# )
# f1_score = 2 * (precision_grid * recall_grid) / (precision_grid + recall_grid)

for model_type in model_colours.keys():
    fig, ax = plt.subplots(1, 1, figsize=(WIDTH*0.5, WIDTH*0.5))
    
#     contours = ax.contour(
#         recall_grid, 
#         precision_grid,
#         f1_score, 
#         levels=np.linspace(0.1, 1, 10), 
#         colors='black',
#         alpha=0.25
#     )
#     ax.clabel(contours, inline=True, fontsize=8, fmt='%.2f')
    
    sns.scatterplot(
        data=df[df['model_type'] == model_type],
        x='val.macro avg.precision',
        y='val.macro avg.recall',
        s=10,
#         alpha=0.5,
        hue='preprocessing.num_gesture_classes',
        style='preprocessing.num_gesture_classes',
        hue_order=['5', '50', '51'],
        style_order=['5', '50', '51'],
        ax=ax,
    edgecolor=None,
    )
    ax.set_xlim((-0.05, 1.05))
    ax.set_ylim((-0.05, 1.05))
    ax.set_xlabel('Precision')
    ax.set_ylabel('Recall')
    ax.plot([0,1], [0,1], color='black', alpha=.1)
    ax.set_title(f'{model_type} $F_1$-score by number of classes')
    ax.legend().set_title('Number of classes')
    
    plt.savefig(
        f'../../report/src/imgs/graphs/05_f1_by_num_gesture_classes_{model_type.lower()}.pdf',
        bbox_inches='tight',
    )

plt.tight_layout()

## Best model by highest lower 90th percentile

In [ ]:
# Calculate the best model
from scipy import stats
type_to_hpars = {
    'CuSUM': ['cusum.thresh'],
    'HMM': ['hmm.covariance_type'],
    'FFNN': [
        'ffnn.dropout_rate',
        'ffnn.l2_coefficient',
        'ffnn.nodes_per_layer.1',
        'ffnn.nodes_per_layer.2',
        'ffnn.nodes_per_layer.3',
        'nn.batch_size',
        'nn.learning_rate',
    ],
    'HFFNN': [
        'hffnn.majority.ffnn.dropout_rate',
        'hffnn.majority.ffnn.l2_coefficient',
        'hffnn.majority.ffnn.nodes_per_layer.1',
        'hffnn.majority.ffnn.nodes_per_layer.2',
        'hffnn.majority.ffnn.nodes_per_layer.3',
        'hffnn.majority.nn.epochs',
        'hffnn.majority.nn.batch_size',
        'hffnn.majority.nn.learning_rate',
        'hffnn.minority.ffnn.dropout_rate',
        'hffnn.minority.ffnn.l2_coefficient',
        'hffnn.minority.ffnn.nodes_per_layer.1',
        'hffnn.minority.ffnn.nodes_per_layer.2',
        'hffnn.minority.ffnn.nodes_per_layer.3',
        'hffnn.minority.nn.epochs',
        'hffnn.minority.nn.batch_size',
        'hffnn.minority.nn.learning_rate',
    ],
    'SVM': ['svm.c', 'svm.class_weight'],
}

def tenth_conf_interval(series):
    mean = np.mean(series)
    sem = stats.sem(series)
    if sem == 0:
        return mean
    confidence_interval = stats.t.interval(
        0.90, 
        len(series) - 1, 
        loc=mean, 
        scale=sem
    )
#     print(mean, sem, confidence_interval)
    return confidence_interval[0]


all_hpars = ['model_type'] + [
    item 
    for sublist in list(type_to_hpars.values()) 
    for item in sublist
]

subset = df[(df['preprocessing.num_gesture_classes'] == '51')]
# print(subset.shape)
gb = subset.groupby(all_hpars, dropna=False)
subset['val.macro avg.f1-score.count'] = (
    gb['val.macro avg.f1-score']
    .transform('count')
)

subset = subset[
    subset['val.macro avg.f1-score.count'].between(5, 100)
]

subset['val.macro avg.f1-score.tenth_conf_interval'] = (
    gb['val.macro avg.f1-score']
    .transform(tenth_conf_interval)
)
subset['val.macro avg.f1-score.mean']  = gb['val.macro avg.f1-score'].transform('mean')
subset['val.macro avg.f1-score.min']   = gb['val.macro avg.f1-score'].transform('min')
subset['val.macro avg.f1-score.max']   = gb['val.macro avg.f1-score'].transform('max')
subset['val.macro avg.f1-score.std']   = gb['val.macro avg.f1-score'].transform('std')
subset['val.macro avg.f1-score.count'] = gb['val.macro avg.f1-score'].transform('count')

subset['group_idx'] = gb.ngroup()
subset = subset.sort_values('val.macro avg.f1-score.tenth_conf_interval')

subset

### Table of fitting times per model/nclasses

In [ ]:
from scipy import stats
from IPython.display import display
type_to_hpars = {
    'CuSUM': ['cusum.thresh'],
    'HMM': ['hmm.covariance_type'],
    'FFNN': [
        'ffnn.dropout_rate',
        'ffnn.l2_coefficient',
        'ffnn.nodes_per_layer.1',
        'ffnn.nodes_per_layer.2',
        'ffnn.nodes_per_layer.3',
        'nn.batch_size',
        'nn.learning_rate',
    ],
    'HFFNN': [
        'hffnn.majority.ffnn.dropout_rate',
        'hffnn.majority.ffnn.l2_coefficient',
        'hffnn.majority.ffnn.nodes_per_layer.1',
        'hffnn.majority.ffnn.nodes_per_layer.2',
        'hffnn.majority.ffnn.nodes_per_layer.3',
        'hffnn.majority.nn.epochs',
        'hffnn.majority.nn.batch_size',
        'hffnn.majority.nn.learning_rate',
        'hffnn.minority.ffnn.dropout_rate',
        'hffnn.minority.ffnn.l2_coefficient',
        'hffnn.minority.ffnn.nodes_per_layer.1',
        'hffnn.minority.ffnn.nodes_per_layer.2',
        'hffnn.minority.ffnn.nodes_per_layer.3',
        'hffnn.minority.nn.epochs',
        'hffnn.minority.nn.batch_size',
        'hffnn.minority.nn.learning_rate',
    ],
    'SVM': ['svm.c', 'svm.class_weight'],
}

all_hpars = ['preprocessing.num_gesture_classes', 'model_type'] + [
    item 
    for sublist in list(type_to_hpars.values()) 
    for item in sublist
]


# print(subset.shape)
gb = df.groupby(all_hpars, dropna=False)
df['group_idx'] = gb.ngroup()

table = (
    df[df['preprocessing.rep_num'] == 4]
    .groupby([
        'model_type', 'preprocessing.num_gesture_classes'
    ])
    ['group_idx']
    .count()
    .reset_index()
    .sort_values(['model_type', 'preprocessing.num_gesture_classes'])
    .pivot(
        columns='model_type', 
        index='preprocessing.num_gesture_classes',
    )
    .T
)
display(table)
display(table.sum())
display(table.T.sum())
display(table.sum().sum())

### Table of fitting times per model/nclasses

In [ ]:
def s_to_hms(s):
#     print(series)
#     s = series['fit_time']
    h = np.floor(s / (60 * 60)).astype(int)
    m = np.floor((s - 60*60*h) / (60)).astype(int)
    s = s - 60*60*h - 60*m
    
    h_str = '' if h == 0 else f'{h: >2}h '
    m_str = '' if m == 0 and h == 0 else f'{m: >2}m '
    s_str = '' if s == 0 and m == 0 and h == 0  else f'{s: >5.2f}s'
    return f'{h_str}{m_str}{s_str}'

display(
    df[df['preprocessing.rep_num'] == 4]
    .groupby([
        'model_type', 'preprocessing.num_gesture_classes'
    ])
    ['fit_time']
    .sum()
    .reset_index()
    .assign(**{
        'fit_time': lambda x: [s_to_hms(xi) for xi in x['fit_time']]
    })
    .sort_values(['model_type', 'preprocessing.num_gesture_classes'])
    .pivot(
        columns='model_type', 
        index='preprocessing.num_gesture_classes',
    ).T
)
# Grouped by model type
display(
    df[df['preprocessing.rep_num'] == 4]
    .groupby(['model_type'])
    ['fit_time']
    .sum()
    .reset_index()
    .assign(**{
        'fit_time': lambda x: [s_to_hms(xi) for xi in x['fit_time']]
    })
)
# Grouped by n gesture classes
display(
    df[df['preprocessing.rep_num'] == 4]
    .groupby(['preprocessing.num_gesture_classes'])
    ['fit_time']
    .sum()
    .reset_index()
    .assign(**{
        'fit_time': lambda x: [s_to_hms(xi) for xi in x['fit_time']]
    })
)

display(s_to_hms(
    df[df['preprocessing.rep_num'] == 4]
    .groupby(['preprocessing.num_gesture_classes'])
    ['fit_time']
    .sum()
    .sum()
))

### Plot models grouped by hyperparameters

In [ ]:
%%script false --no-raise-error # This cell is unused
fig, ax = plt.subplots(1, 1, figsize=(WIDTH, 5))

order = subset.sort_values(
    'val.macro avg.f1-score.tenth_conf_interval'
)['group_idx'].unique()

sns.stripplot(
    data=subset,
    y='val.macro avg.f1-score',
    x='group_idx',
    hue='model_type',
#     legend=False,
    order=order,
    size=2,
#     alpha=0.5,
    ax=ax,
)

sns.pointplot(
    data=subset, 
    x="group_idx", 
    y="val.macro avg.f1-score", 
    hue="model_type",
    linestyle="none", 
    errorbar=None,
    marker="_", 
    markersize=2, 
    palette='dark:black',
    markeredgewidth=1,
    zorder=10,
    ax=ax,
)

ax.set(
    title='$F_1$ score for each set of hyperparameters, by model type',
    xlabel='Hyperparameter index',
    ylabel='$F_1$-score',
    ylim=((-0.05, 1.05)),
    xticks=[],
)

vis.add_grid(ax)
# ax.set_xticks(ax.get_xticks())
# ax.set_xticklabels(
#     order,
#     rotation=90,
# )


# vis.fmt_legend(ax)
# handles, labels = ax.get_legend_handles_labels()
# ax.add_artist(plt.legend(handles[:-5], labels[:-5], title="Model Type"))

handles, labels = ax.get_legend_handles_labels()
ax.legend(
    handles=handles[:-5],
    labels=labels[:-5],
    borderpad=0.5,
    labelspacing=0.1,
    handlelength=0.1,
    title='Model Type',
    loc='upper left',
)


plt.savefig(
    '../../report/src/imgs/graphs/05_all_f1_scores.pdf',
    bbox_inches='tight',
)

plt.show()

In [ ]:

fig, ax = plt.subplots(1, 1, figsize=(WIDTH, 2.5))

order = subset.sort_values(
    'val.macro avg.f1-score.tenth_conf_interval',
    ascending=False,
)['group_idx'].unique()[:50][::-1]

data = subset[subset['group_idx'].isin(order)]

sns.stripplot(
    data=data,
    y='val.macro avg.f1-score',
    x='group_idx',
    hue='model_type',
    order=order,
    hue_order=list(model_colours.keys()),
#     size=4,
    **stripplot_kwargs,
    jitter=False,
#     alpha=0.5,
    ax=ax,
)

sns.pointplot(
    data=data, 
    x="group_idx", 
    y="val.macro avg.f1-score", 
    hue="model_type",
    linestyle="none", 
    errorbar=None,
    marker="_", 
    markersize=4, 
    palette='dark:black',
    markeredgewidth=1,
    zorder=10,
    alpha=0.5,
    ax=ax,
)

ax.set(
    title=f'$F_1$-score for the top {len(order)} hyperparameters\n'
    '(ordered by the lower bound of the 90\% CI)',
    xlabel='Hyperparameter index',
    ylabel='$F_1$-score',
)
ax.yaxis.grid(True)
ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(
    order,
    rotation=90,
)
yticks = np.linspace(
    data['val.macro avg.f1-score'].min() * 0.95,
    data['val.macro avg.f1-score'].max() / 0.95,
    10,
)
ax.set(
    yticks=yticks,    
    yticklabels=np.round(yticks, 2),    
)
# handles, labels = ax.get_legend_handles_labels()
# ax.add_artist(plt.legend(handles[:5], labels[:5], title="Model Type"))

handles, labels = ax.get_legend_handles_labels()
ax.legend(
    handles=handles[:5],
    labels=labels[:5],
    borderpad=0.5,
    labelspacing=0.1,
    handlelength=0.1,
    title='Model Type',
#     **kwargs,
)


plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_best_hpar_comparison.pdf',
    bbox_inches='tight',
)
plt.show()

In [ ]:
print("Best model:")
from pprint import pprint
pprint(
    subset
    .sort_values('val.macro avg.f1-score.tenth_conf_interval', ascending=False)
    .groupby('group_idx')
    .tail(1)
#     [type_to_hpars['FFNN']]
    .head(1)
    .squeeze()
    .to_dict()
)

In [ ]:
model_type = 'FFNN'

def prettify_col(col):
    return {
        'val.macro avg.f1-score.mean': '$F_1$-score Mean',
        'val.macro avg.f1-score.std': '$F_1$-score Std.Dev.',
        'group_idx': 'Index',
        'cusum.thresh': 'Threshold',
        'ffnn.dropout_rate': 'Dropout Rate',
        'ffnn.l2_coefficient': 'L2 Coefficient',
        'ffnn.l2_coefficient.log10': '$\log_{10}(\text{L2 Coefficient})$',
        'ffnn.nodes_per_layer.1': 'Nodes (layer 1)',
        'ffnn.nodes_per_layer.2': 'Nodes (layer 2)',
        'ffnn.nodes_per_layer.3': 'Nodes (layer 3)',
        'hffnn.majority.ffnn.dropout_rate': 'Majority: Dropout Rate',
        'hffnn.majority.ffnn.l2_coefficient': 'Majority: L2 Coefficient',
        'hffnn.majority.ffnn.nodes_per_layer.1': 'Majority: Nodes (layer 1)',
        'hffnn.majority.ffnn.nodes_per_layer.2': 'Majority: Nodes (layer 2)',
        'hffnn.majority.ffnn.nodes_per_layer.3': 'Majority: Nodes (layer 3)',
        'hffnn.majority.nn.batch_size': 'Majority: Batch Size',
        'hffnn.majority.nn.epochs': 'Majority: Epochs',
        'hffnn.majority.nn.learning_rate': 'Majority: Learning Rate',
        'hffnn.majority.nn.optimizer': 'Majority: Optimizer',
        'hffnn.minority.ffnn.dropout_rate': 'Minority: Dropout Rate',
        'hffnn.minority.ffnn.l2_coefficient': 'Minority: L2 Coefficient',
        'hffnn.minority.ffnn.nodes_per_layer.1': 'Minority: Nodes (layer 1)',
        'hffnn.minority.ffnn.nodes_per_layer.2': 'Minority: Nodes (layer 2)',
        'hffnn.minority.ffnn.nodes_per_layer.3': 'Minority: Nodes (layer 3)',
        'hffnn.minority.nn.batch_size': 'Minority: Batch Size',
        'hffnn.minority.nn.epochs': 'Minority: Epochs',
        'hffnn.minority.nn.learning_rate': 'Minority: Learning Rate',
        'hffnn.minority.nn.optimizer': 'Minority: Optimizer',
        'hmm.covariance_type': 'Covariance Type',
        'nn.batch_size': 'Batch Size',
        'nn.batch_size.log10': '$\log_{10}(\text{Batch Size})$',
        'nn.epochs': 'Epochs',
        'nn.learning_rate': 'Learning Rate',
        'nn.learning_rate.log10': '$\log_{10}(\text{Learning Rate})$',
        'nn.optimizer': 'Optimizer',
        'svm.c': 'C',
        'svm.class_weight': 'Class Weight',
    }.get(col, col)

def df_to_latex(df, model_type):
    path = f'../../report/src/tables/05_best_{model_type.lower().replace(" ", "_")}_hpars.generated.tex'
    print('DONT FORGET TO UPDATE LaTeX tables: ', path)
    df.to_latex(
        path,
        caption=f'Top {len(df)} performing {model_type} hyperparameter combinations, ordered by '
                f'the lower bound of the 90 percent confidence interval for $F_1$-score.',
        label=f'tab:05_best_{model_type.lower().replace(" ", "_")}_hpars',
        index=False,
        float_format=lambda x: '%.3e' % x,
        na_rep='-'
    )

for model_type in type_to_hpars.keys():
    latex_df = (
        subset[
            subset['model_type'] == model_type
        ]
        .sort_values('val.macro avg.f1-score.tenth_conf_interval', ascending=False)
        .groupby('group_idx')
        .tail(1)
        [
            ['group_idx', 'val.macro avg.f1-score.mean', 'val.macro avg.f1-score.std'] 
            + type_to_hpars[model_type]
        ]
        .head(10)
        .reset_index(drop=True)
        .rename(columns=prettify_col)
        .replace({
            'tied': 'Tied',
            'spherical': 'Spherical',
            'diag': 'Diagonal',
            'full': 'Full',
            'balanced': 'Balanced',
        } | {} if model_type != 'SVM' else {
            np.nan: 'Unbalanced',
        } | {} if model_type not in ('HFFNN', 'FFNN') else {
            np.nan: 'None',
        })
    )
    display(latex_df)
    
    if model_type == 'HFFNN':
        majority = latex_df[[c for c in latex_df.columns if 'Minority' not in c]]
        minority = latex_df[[c for c in latex_df.columns if 'Majority' not in c]]
        majority.columns = [c.replace("Majority: ", "") for c in majority.columns]
        minority.columns = [c.replace("Minority: ", "") for c in minority.columns]
        df_to_latex(majority, 'Majority HFFNN')
        df_to_latex(minority, 'Minority HFFNN')
    else:
        df_to_latex(latex_df, model_type)
    

### Model types grouped by hpar index

In [ ]:
from matplotlib.gridspec import GridSpec
fig = plt.figure(figsize=(WIDTH, 7))
gs = GridSpec(4, 2, figure=fig)
ax1 = fig.add_subplot(gs[0, :])
ax2 = fig.add_subplot(gs[1, :])
ax3 = fig.add_subplot(gs[2, :])
ax4 = fig.add_subplot(gs[3, 0])
ax5 = fig.add_subplot(gs[3, 1])

model_axs = {
    'FFNN':  ax1,
    'HFFNN': ax2,
    'SVM':   ax3,
    'HMM':   ax4,
    'CuSUM': ax5,
}

for model_type, color in model_colours.items():
    ax = model_axs[model_type]
    data = subset[subset['model_type'] == model_type]
    
    order = data.sort_values(
        'val.macro avg.f1-score.tenth_conf_interval'
    )['group_idx'].unique()

    sns.stripplot(
        data=data,
        y='val.macro avg.f1-score',
        x='group_idx',
#         hue='model_type',
        color=color,
        order=order,
        **(stripplot_kwargs | dict(
            size=2.5
        )),
#         alpha=0.5,
        ax=ax,
        jitter=False,
        legend=False,
    )

    sns.pointplot(
        data=data, 
        x="group_idx", 
        y="val.macro avg.f1-score", 
        hue="model_type",
        linestyle="none", 
        errorbar=None,
        marker="_", 
        markersize=2.5 if model_type in ['HFFNN', 'FFNN'] else 5,
        palette='dark:black',
        markeredgewidth=1,
        zorder=10,
        ax=ax,
        legend=False,
    )
    
    ax.set(
        title=f'{model_type} $F_1$-scores',
        xlabel='Hyperparameter index',
        ylabel='$F_1$-score',
        ylim=((-0.05, 1.05)),
        yticks=[0.0, 0.2, 0.4, 0.6, 0.8, 1.0],
    )
    ax.yaxis.grid(True)
#     ax.set_xticks(ax.get_xticks())
#     ax.set_xticklabels(order, rotation=90)

    xticks = ax.get_xticks()
    if len(xticks) > 10:
        ax.set_xticks([xticks[i] for i in range(len(xticks)) if i % 8 == 0])

plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_hpar_comparison_per_model_type.pdf',
    bbox_inches='tight',
)
plt.show()

## ECDF for all model types

In [ ]:
%%script false --no-raise-error # This cell is unused
fig, ax = plt.subplots(1, 1, figsize=(WIDTH, WIDTH))
for model_type, color in model_colours.items():
    scores = df.loc[
        (df['preprocessing.num_gesture_classes'] == '51')
        & (df['model_type'] == model_type),
        'val.macro avg.f1-score'
    ]
    sorted_scores = np.sort(scores)
    ecdf = np.arange(1, len(sorted_scores) + 1) / len(sorted_scores)
    ax.plot(ecdf, sorted_scores, color=color, label=model_type)
plt.legend()
ax.set(
    title='ECDF for all model types',
    ylabel='$F_1$-score',
    xlabel='Cumulative Probability',
)
plt.show()

## Training vs Inference times

In [ ]:
# Three plots showing the precision and recall for all models.
# Each plot showing either 51, 50, or 5 gesture classes
fig, axs = plt.subplots(1, 2, figsize=(WIDTH, WIDTH*.5))

df['val.pred_time_per_obs.log10'] = np.log10(df['val.pred_time_per_obs'])
df['trn.pred_time_per_obs.log10'] = np.log10(df['trn.pred_time_per_obs'])
df['fit_time_per_obs.log10'] = np.log10(df['fit_time_per_obs'])

sns.scatterplot(
    data=df[df['preprocessing.num_gesture_classes'] == '51'],
    x='val.pred_time_per_obs.log10',
    y='trn.pred_time_per_obs.log10',
#     s=5,
#     edgecolor=None,
#     linewidth=0.5,
    **scatterplot_kwargs,
    hue='model_type',
    hue_order=list(model_colours.keys()),
    ax=axs[0],
)

sns.scatterplot(
    data=df[
        (df['preprocessing.num_gesture_classes'] == '51')
    ],
    x='fit_time_per_obs.log10',
    y='trn.pred_time_per_obs.log10',
#     s=5,
#     edgecolor=None,
#     linewidth=0.5,
    legend=False,
    **scatterplot_kwargs,
    hue='model_type',
    hue_order=list(model_colours.keys()),
    ax=axs[1],
)

axs[0].set(
    title='Validation vs Training Inference Time',
    xlabel='Validation inference time\n($\log_{10}$ seconds/observation)',
    ylabel='Training inference time\n($\log_{10}$ seconds/observation)',
    yticks=np.arange(-6, 0.1, 1),
    xticks=np.arange(-6, 0.1, 1),
)
axs[1].set(
    title='Fitting vs inference time',
    xlabel='Fitting time\n($\log_{10}$ seconds/observation)',
#     ylabel='Inference Time\n($\log_{10}$ seconds/observation)',
    ylabel=None,
    yticks=np.arange(-6, 0.1, 1),
    xticks=np.arange(-4, -0.9, 0.5),
    yticklabels=[],
)

# for ax in axs.flatten():
# axs[0].legend().set_title("Model Type")
vis.fmt_legend(axs[0], title='Model Type')

vis.add_grid(axs[0])
vis.add_grid(axs[1])
plt.tight_layout()

plt.savefig(
    '../../report/src/imgs/graphs/05_inf_trn_times_per_obs.pdf',
    bbox_inches='tight'
)

## Inference time vs $F_1$

In [ ]:
%%script false --no-raise-error # This cell is unused
fig, ax = plt.subplots(1, 1, figsize=(WIDTH*0.5, WIDTH*0.5))

sns.scatterplot(
    data=df[df['preprocessing.num_gesture_classes'] == '51'],
    x='val.pred_time_per_obs.log10',
    y='val.macro avg.f1-score',
    s=5,
    edgecolor=None,
    linewidth=0.1,
    hue='model_type',
    hue_order=list(model_colours.keys()),
    ax=ax,
)

ax.set_ylim((-0.05, 1.05))
ax.set_ylabel('$F_1$-score')
ax.set_xlabel('Inference time\n($\log_{10}$ seconds/observation)')
ax.set_title('Inference time per observation against $F_1$ score')
ax.legend().set_title("Model Type")

vis.add_grid(ax)
plt.tight_layout()

plt.savefig(
    '../../report/src/imgs/graphs/05_inference_time_per_obs_vs_f1.pdf', 
    bbox_inches='tight'
)

## Mean Confidence Matrices

### 5 Gesture classes

In [ ]:
%%script false --no-raise-error
nclasses = '5'
data = df[
    (df['preprocessing.num_gesture_classes'] == nclasses)
]
conf_mats = {}
conf_mat_totals = {}

hpar = 'model_type'

assert len(data[hpar].unique()) < 10

for i, row in data.iterrows():
    y_true, y_pred = get_npz_data_from_model('../' + row['model_dir'])
    cm = tf.math.confusion_matrix(
        y_true.flatten(), 
        y_pred.flatten()
    ).numpy()
    f1_score = sklearn.metrics.f1_score(
        y_true.flatten(), 
        y_pred.flatten(),
        average='macro',
        zero_division=0,
    )
    hpar_item = row[hpar]
    
    if hpar_item in conf_mats:
        conf_mats[hpar_item] += cm.astype(float) * f1_score
        conf_mat_totals[hpar_item] += f1_score
    else:
        conf_mats[hpar_item] = cm.astype(float) * f1_score
        conf_mat_totals[hpar_item] = f1_score

# NOTE: Regular WIDTH is used or else LaTeX throws it's toys out the cot
fig, axs = plt.subplots(
    2,2,
    figsize=(WIDTH, WIDTH)
)
print(conf_mats.keys())
axs = axs.flatten()
for i, (hpar_item, conf_mat) in enumerate(conf_mats.items()):
#     conf_mat[-1, -1] = 0
#     conf_mat /= conf_mat_totals[hpar_item]
#     conf_mat /= conf_mat.max()
#     print(conf_mat_totals[hpar_item])
    # Normalize the matrix manually to get nicer annotations
    vis.conf_mat(conf_mat / conf_mat.sum(axis=0), ax=axs[i], norm=None)
    axs[i].set_title(
        f'{hpar_item} Confusion Matrices\n{nclasses} classes'
    )

plt.tight_layout()
plt.savefig(
    f'../../report/src/imgs/graphs/05_mean_conf_mat_{nclasses}_classes.pdf',
    bbox_inches='tight',
)

### 50 Gesture classes

In [ ]:
%%script false --no-raise-error
nclasses = '50'
data = df[
    (df['preprocessing.num_gesture_classes'] == nclasses)
]
conf_mats = {}
conf_mat_totals = {}

hpar = 'model_type'

assert len(data[hpar].unique()) < 10

for i, row in data.iterrows():
    y_true, y_pred = get_npz_data_from_model('../' + row['model_dir'])
    cm = tf.math.confusion_matrix(
        y_true.flatten(), 
        y_pred.flatten()
    ).numpy()
    f1_score = sklearn.metrics.f1_score(
        y_true.flatten(), 
        y_pred.flatten(),
        average='macro',
        zero_division=0,
    )
    hpar_item = row[hpar]
    
    if hpar_item in conf_mats:
        conf_mats[hpar_item] += cm.astype(float) * f1_score
        conf_mat_totals[hpar_item] += f1_score
    else:
        conf_mats[hpar_item] = cm.astype(float) * f1_score
        conf_mat_totals[hpar_item] = f1_score

fig, axs = plt.subplots(
    2, 2,
    figsize=(WIDTH, WIDTH)
)
print(conf_mats.keys())
axs = axs.flatten()
for i, (hpar_item, conf_mat) in enumerate(conf_mats.items()):
#     conf_mat /= conf_mat.max()
    vis.conf_mat(conf_mat / conf_mat.sum(axis=0), ax=axs[i], norm=None)
#     vis.conf_mat(conf_mat, ax=axs[i])
    axs[i].set_title(
        f'{hpar_item} Confusion Matrices\n{nclasses} classes'
    )

plt.tight_layout()
plt.savefig(
    f'../../report/src/imgs/graphs/05_mean_conf_mat_{nclasses}_classes.pdf',
    bbox_inches='tight',
)

### 51 Gesture classes

In [ ]:
%%script false --no-raise-error # This cell is unused
# NOTE: skipping this cell might have been a mistake
nclasses = '51'
data = df[
    (df['preprocessing.num_gesture_classes'] == nclasses)
]
conf_mats = {}
conf_mat_totals = {}

hpar = 'model_type'

assert len(data[hpar].unique()) < 10

for i, row in data.iterrows():
    try:
        y_true, y_pred = get_npz_data_from_model('../' + row['model_dir'])
    except FileNotFoundError:
        y_true, y_pred = get_npz_data_from_model('./' + row['model_dir'])
    cm = tf.math.confusion_matrix(
        y_true.flatten(), 
        y_pred.flatten()
    ).numpy()
    f1_score = sklearn.metrics.f1_score(
        y_true.flatten(), 
        y_pred.flatten(),
        average='macro',
        zero_division=0,
    )
    hpar_item = row[hpar]
    
    if hpar_item in conf_mats:
        conf_mats[hpar_item] += cm.astype(float) * f1_score
        conf_mat_totals[hpar_item] += f1_score
    else:
        conf_mats[hpar_item] = cm.astype(float) * f1_score
        conf_mat_totals[hpar_item] = f1_score
    if i % 100 == 0:
        print(i, end=' ', flush=True)

In [ ]:
%%script false --no-raise-error
fig, axs = plt.subplots(
    3, 2,
    figsize=(WIDTH, WIDTH)
)
axs = axs.flatten()
for i, (hpar_item, conf_mat) in enumerate(conf_mats.items()):
    vis.conf_mat(conf_mat / conf_mat.sum(axis=0), ax=axs[i], norm=None)
    axs[i].set_title(
        f'{hpar_item} Confusion Matrices\n{nclasses} classes'
    )
axs[-1].axis('off')

plt.tight_layout()
plt.savefig(
    f'../../report/src/imgs/graphs/05_mean_conf_mat_{nclasses}_classes.pdf',
    bbox_inches='tight',
)

## Conf Mats for each num-gesture-class with precision-recall plot

In [ ]:
# recall_grid, precision_grid = np.meshgrid(
#     np.linspace(0, 1, 100), 
#     np.linspace(0, 1, 100)
# )
# f1_score_grid = 2 * (precision_grid * recall_grid) / (precision_grid + recall_grid)


for model_type in ('FFNN', 'HFFNN', 'SVM', 'CuSUM', 'HMM'):
    data = df[
        (df['model_type'] == model_type)
    ]
    conf_mats = {}
    conf_mat_totals = {}

    hpar = 'preprocessing.num_gesture_classes'

    assert len(data[hpar].unique()) < 10
    print(model_type, flush=True)

    for i, row in data.sort_values(hpar).iterrows():
        try:
            y_true, y_pred = get_npz_data_from_model('../' + row['model_dir'])
        except FileNotFoundError:
            y_true, y_pred = get_npz_data_from_model('./' + row['model_dir'])
        cm = tf.math.confusion_matrix(
            y_true.flatten(), 
            y_pred.flatten()
        ).numpy()
        f1_score = sklearn.metrics.f1_score(
            y_true.flatten(), 
            y_pred.flatten(),
            average='macro',
            zero_division=0,
        )
        hpar_item = row[hpar]

        if hpar_item in conf_mats:
            conf_mats[hpar_item] += cm.astype(float) * f1_score
            conf_mat_totals[hpar_item] += f1_score
        else:
            conf_mats[hpar_item] = cm.astype(float) * f1_score
            conf_mat_totals[hpar_item] = f1_score
        if i % 50 == 0:
            print(i, end=' ', flush=True)

    if model_type == 'HFFNN':
        fig, axs = plt.subplots(
            1, 2,
            figsize=(WIDTH, WIDTH*.5),
            squeeze=False,
        )
    else:
        fig, axs = plt.subplots(
            2, 2,
            figsize=(WIDTH, WIDTH),
            squeeze=False,
        )
    axs = axs.flatten()

    for i, (hpar_item, conf_mat) in enumerate(conf_mats.items()):
        cmat = conf_mat / conf_mat_totals[hpar_item]
        vis.conf_mat(cmat, ax=axs[i])
        axs[i].set_title(
            f'Weighted Confusion Matrix\n'
            f'{model_type} {hpar_item}-class'
        )

#     contours = axs[-1].contour(
#         recall_grid, 
#         precision_grid,
#         f1_score_grid, 
#         levels=np.linspace(0.1, 1, 10), 
#         colors='black',
#         alpha=0.25
#     )
#     axs[-1].clabel(contours, inline=True, fontsize=8, fmt='%.2f')
    
    sns.scatterplot(
        data=df[df['model_type'] == model_type],
        x='val.macro avg.precision',
        y='val.macro avg.recall',
#         s=5,
#         alpha=0.5,
        hue='preprocessing.num_gesture_classes',
        style='preprocessing.num_gesture_classes',
        hue_order=['5', '50', '51'],
        style_order=['5', '50', '51'],
        ax=axs[-1],
        **scatterplot_kwargs,
#         edgecolor=None,
    )
    axs[-1].set(
        xlim=(-0.05, 1.05),
        ylim=(-0.05, 1.05),
        xlabel='Precision',
        ylabel='Recall',
        title=f'Precision-recall plot\n{model_type}s by \#Classes',
    )
    vis.add_grid(axs[-1])
#     axs[-1].plot([0,1], [0,1], color='black', alpha=.25)
#     axs[-1].legend().set_title('Classes')
    vis.fmt_legend(axs[-1], title='Classes')

    plt.tight_layout()
    plt.savefig(
        f'../../report/src/imgs/graphs/05_mean_conf_mat_{model_type.lower()}.pdf',
        bbox_inches='tight',
    )
    plt.show()

## Regularization Plots

In [ ]:
%%script false --no-raise-error
n_gesture_classes = (
    '5',
    '50',
    '51',
)
# TODO this ratio might need to be changed
fig, axs = plt.subplots(
    3, 2, 
    figsize=(WIDTH, WIDTH/3.)
)

for i, ngestures in enumerate(n_gesture_classes):
    data = df[df['preprocessing.num_gesture_classes'] == ngestures]
    sns.scatterplot(
        data=data,
        x='ffnn.l2_coefficient',
        y='ratio.macro avg.f1-score',
        ax=axs[i, 0],
        edgecolor=None,
    )
    sns.scatterplot(
        data=data,
        x='ffnn.dropout_rate',
        y='ratio.macro avg.f1-score',
        ax=axs[i, 1],
        edgecolor=None,
    )
    axs[i, 0].set_title(f'{ngestures} gestures')
    axs[i, 1].set_title(f'{ngestures} gestures')

plt.suptitle("$F_1$-ratio against regularisation")
plt.tight_layout()

## Ratio $F_1$ scores vs actual $F_1$ scores

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(WIDTH, WIDTH))

sns.scatterplot(
    data=df[df['preprocessing.num_gesture_classes'] == '51'],
    x='val.macro avg.f1-score',
    y='trn.macro avg.f1-score',
#     s=5,
    hue='model_type',
    hue_order=list(model_colours.keys()),
#     alpha=0.5,
    ax=axs[0, 0],
    **scatterplot_kwargs,
#     edgecolor=None,
)
axs[0, 0].set_xlim((-0.05, 1.05))
axs[0, 0].set_ylim((-0.05, 1.05))
axs[0, 0].plot([0,1], [0,1], color='black', alpha=.1)
axs[0, 0].set_title("Training vs Val. $F_1$ score\n")
axs[0, 0].set_xlabel("Validation $F_1$")
axs[0, 0].set_ylabel("Training $F_1$")
axs[0, 0].legend().set_title('Model Type')

sns.scatterplot(
    data=df[df['preprocessing.num_gesture_classes'] == '51'],
    x='val.macro avg.f1-score',
    y='trn.macro avg.f1-score',
#     s=5,
    hue='model_type',
    hue_order=list(model_colours.keys()),
#     alpha=0.5,
    ax=axs[0, 1],
    legend=False,
    **scatterplot_kwargs,
#     edgecolor=None,
)
axs[0, 1].set_xlim((0.45, 1.05))
axs[0, 1].set_ylim((0.5, 1.05))
axs[0, 1].plot([0,1], [0,1], color='black', alpha=.1)
axs[0, 1].set_title("Training vs Val. $F_1$ score\n(magnified)")
axs[0, 1].set_xlabel("Validation $F_1$")
axs[0, 1].set_ylabel("Training $F_1$")
# axs[0, 1].legend().set_title('Model Type')
axs[0, 1].set_xticks(np.arange(0.5, 1.01, 0.1))

sns.scatterplot(
    data=df[df['preprocessing.num_gesture_classes'] == '51'],
    x='val.macro avg.f1-score',
    y='ratio.macro avg.f1-score',
#     s=5,
    hue='model_type',
    hue_order=list(model_colours.keys()),
#     alpha=0.5,
    legend=False,
    ax=axs[1, 0],
#     edgecolor=None,
    **scatterplot_kwargs,
)
axs[1, 0].set_xlim((-0.05, 1.05))
axs[1, 0].set_title("$F_1$-ratio vs $F_1$-score\n")
axs[1, 0].set_xlabel("Val. $F_1$")
axs[1, 0].set_ylabel(r"$F_1$-ratio ($\frac{Training}{Val.}$)")
# axs[1, 0].legend().set_title('Model Type')
axs[1, 0].plot([0,1], [1, 1], color='black', alpha=.1)

sns.scatterplot(
    data=df[
        (df['preprocessing.num_gesture_classes'] == '51')
        & (df['val.macro avg.f1-score'] >= 0.5)
    ],
    x='val.macro avg.f1-score',
    y='ratio.macro avg.f1-score',
#     s=5,
    hue='model_type',
    hue_order=list(model_colours.keys()),
#     alpha=0.5,
    legend=False,
    ax=axs[1, 1],
#     edgecolor=None,
    **scatterplot_kwargs,
)
axs[1,1].set(
    xlim=(0.45, 1.05),
    xticks=np.arange(0.5, 1.01, 0.1),
    title="$F_1$-ratio vs $F_1$-score\n(magnified)",
    xlabel="Val. $F_1$",
    ylabel=r"$F_1$-ratio ($\frac{Training}{Val.}$)",
)

axs[1, 1].plot([0,1], [1, 1], color='black', alpha=.1)

vis.fmt_legend(axs[0, 0], title='Model Type')
for ax in axs.flat:
    vis.add_grid(ax)
plt.tight_layout()

plt.savefig(
    '../../report/src/imgs/graphs/05_f1_vs_f1_ratio.pdf'
)

## Training/validation loss ratios

In [ ]:
%%script false --no-raise-error # This cell is unused
fig, axs = plt.subplots(1, 2, figsize=(WIDTH, WIDTH*.5))

sns.scatterplot(
    data=df[
        (df['preprocessing.num_gesture_classes'] == '51')
        & (df['model_type'] == 'FFNN')
    ],
    y='val.loss',
    x='trn.loss',
    color='tab:blue',
    s=20,
    alpha=0.5,
    ax=axs[0],
    edgecolor=None,
)


sns.scatterplot(
    data=df[
        (df['preprocessing.num_gesture_classes'] == '51')
        & (df['model_type'] == 'FFNN')
    ].assign(**{
        'ratio.loss': lambda x: x['trn.loss'] / x['val.loss']
    }),
    x='ratio.loss',
    y='val.loss',
    color='tab:blue',
    s=20,
    alpha=0.5,
    ax=axs[1],
    edgecolor=None,
)


axs[0].set_ylim((-0.05, 3.6))
axs[1].set_ylim((-0.05, 3.6))

axs[1].plot([1, 1], [0, axs[1].get_ylim()[1]], color='black', alpha=.1)

min_max = min(axs[0].get_xlim()[1], axs[0].get_ylim()[1] )
axs[0].plot([0, min_max], [0, min_max], color='black', alpha=.1)

axs[0].set(
    title='Validation loss vs training loss\n(FFNN only)',
    xlabel='Training Loss',
    ylabel='Validation Loss',
)
axs[1].set(
    title='Validation loss vs loss ratio\n(FFNN only)',
    xlabel=r'Loss ratio ($\frac{Training}{Validation}$)',
    ylabel='Validation Loss',
)

plt.tight_layout()

plt.savefig(
    '../../report/src/imgs/graphs/05_val_trn_loss_ratios.pdf'
)

print("TODO: The training and validation loss aren't comparable because the training loss is weighed but validation loss is not.")
print("TODO: also not comparable because of dropout")

## Precision vs Recall plots for each model individually

In [ ]:
%%script false --no-raise-error
for model_type, color in model_colours.items():
    data = df[
        (df['preprocessing.num_gesture_classes'] == '51')
        & (df['model_type'] == model_type)
    ]

    fig, ax = plt.subplots(1, 1, fisize=(WIDTH, WIDTH))
    p_min = data['val.macro avg.precision'].min() / 1.10
    p_max = data['val.macro avg.precision'].max() * 1.10
    r_min = data['val.macro avg.recall'].min()    / 1.10
    r_max = data['val.macro avg.recall'].max()    * 1.10
    print(f"{p_min=}, {p_max=}, {r_min=}, {r_max=}")

    sns.scatterplot(
        data=data,
        y='val.macro avg.recall',
        x='val.macro avg.precision',
        color=color,
        alpha=0.5,
        s=20,
        ax=ax,
    edgecolor=None,
    )

    p_range = p_max - p_min
    r_range = r_max - r_min
    ax.set(
        title=f"Precision vs Recall For {model_type} models",
        xlabel='Precision',
        ylabel='Recall',
        xlim=(p_min - p_range*0.025, p_max + p_range*0.025),
        ylim=(r_min - r_range*0.025, r_max + r_range*0.025),
    )

    plt.savefig(
        f'../../report/src/imgs/graphs/05_in_depth_{model_type.lower()}_p_vs_r.pdf',
        bbox_inches='tight',
    )
    plt.show()

## In depth FFNN plots

In [ ]:
data = df[
    (df['preprocessing.num_gesture_classes'] == '51') 
    & (df['model_type'] == 'FFNN')
]
hyperpars = [
    'nn.learning_rate.log10',
    'nn.batch_size.log10',
    'ffnn.num_layers',
    'ffnn.nodes_per_layer.1.log10',
    'ffnn.nodes_per_layer.2.log10',
    'ffnn.nodes_per_layer.3.log10',
    'ffnn.nodes_per_layer.-1.log10',
    'ffnn.l2_coefficient.log10',
    'ffnn.dropout_rate',
]
data['Recall $>$ 0.7'] = data['val.macro avg.recall'] > 0.7

data['ffnn.nodes_per_layer.1.log10'].fillna(0, inplace=True)
data['ffnn.nodes_per_layer.2.log10'].fillna(0, inplace=True)
data['ffnn.nodes_per_layer.3.log10'].fillna(0, inplace=True)
data['ffnn.nodes_per_layer.-1.log10'].fillna(0, inplace=True)


### Precision recall histogram

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(WIDTH, 2))
for ax, nlayers in zip(axs, [1,2,3]):
    sns.histplot(
        data=data[data['ffnn.num_layers'] == nlayers],
        x='val.macro avg.precision',
        y='val.macro avg.recall',
        binwidth=(0.05, 0.05),
        binrange=(
            (0.0, 1.0),
            (0.0, 1.0)
        ),
        cmap='Spectral',
        vmin=0,
        vmax=160,
        ax=ax,
        cbar=nlayers == 3,
    )
    vis.add_grid(ax)
    
    vis.rename_ax(ax)
    ax.set(
        xlim=(-0.05, 1.05),
        ylim=(-0.05, 1.05),
        xticks=np.arange(0, 1.01, 0.25),
        title=f'{nlayers}-layer FFNNs',
        yticks=np.arange(0, 1.01, 0.25),
    )
    if nlayers != 1:
        ax.set(
            yticklabels=[],
            ylabel=None,
        )
        

plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,x=p,y=r,c=nlayers,histplot.pdf',
    bbox_inches='tight',
)
plt.show()

### x=precision y=recall hue=nlayers

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(WIDTH, WIDTH*0.5))
sns.scatterplot(
    data=data,
    x='val.macro avg.precision',
    y='val.macro avg.recall',
    hue='ffnn.num_layers',
#     s=2.5,
    **scatterplot_kwargs,
#     edgecolor='#0005',
#     alpha=0.75,
    legend=False,
    palette=['tab:blue', 'tab:orange', 'tab:green'],
    ax=axs[1]
)
# vis.fmt_legend(axs[0])
vis.add_grid(axs[1])
axs[1].set(
    xticks=np.arange(0, 1.01, 0.2),
    yticks=np.arange(0, 1.01, 0.2),
)

sns.stripplot(
    data=data,
    y='val.macro avg.f1-score',
    x='ffnn.num_layers',
    hue='ffnn.num_layers',
    palette=['tab:blue', 'tab:orange', 'tab:green'],
    legend=False,
#     s=2,
    jitter=0.3,
#     alpha=0.5, 
    ax=axs[0],
    native_scale=True,
    **stripplot_kwargs,
#     edgecolor='#0005',
#     linewidth=0.5
)
axs[0].set(
    xlim=(0, 4),
    xticks=[1, 2, 3],
    yticks=np.arange(0, 1.01, 0.2),
)
vis.add_grid(axs[0])

vis.rename_axs(axs)

plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,x=p,y=r,h=nlayers.pdf',
    bbox_inches='tight',
)
plt.show()


### x=hpar y=recall, all hyperparameters

In [ ]:
aspect = 2
col_wrap = 3
g = sns.catplot(
    data=vis.add_jitter(data, hyperpars, amount=0.025).melt( 
        id_vars=['Recall $>$ 0.7'], 
        value_vars=hyperpars,
    ), 
    x="value", 
    col="variable",
    col_wrap=col_wrap,
    hue="Recall $>$ 0.7",
    hue_order=[True, False],
    kind="strip",
    sharex=False,
    dodge=True,
    **stripplot_kwargs,
#     size=4,
    jitter=0.25,
#     alpha=0.1,
    height=WIDTH / col_wrap / aspect,
    legend=False,
    aspect=aspect,
)

for ax in g.axes.flat:
    hpar = ax.get_title().split(' = ')[1]
    ax.set(
        xlabel=f'{rename_hpars[hpar]}',
        title=None,
        ylabel='Recall $<$ 0.7?'
    )
    vis.add_grid(ax)
    
plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,c=hpar,x=hpar,y=recall>70.pdf',
    bbox_inches='tight',
)

### x=LR y=Nodes layer 1

In [ ]:
# #Nodes layer 1 vs LR has something happening
fig, ax = plt.subplots(1, 1, figsize=(WIDTH, 0.5*WIDTH))
sns.scatterplot(
    data=vis.add_jitter(data, hyperpars, amount=0.025),
    y='ffnn.nodes_per_layer.1.log10',
    x='nn.learning_rate.log10',
    hue='val.macro avg.f1-score',
    **scatterplot_kwargs,
#     s=5,
#     edgecolor='#0005',
    palette='Spectral',
    hue_norm=(0, 1),
    ax=ax,
    legend=False,
)
vis.rename_ax(ax)
ax.set(
    xticks=np.arange(-6, -0.9, 0.5),
    yticks=np.arange(0.5, 3, 0.25),
    xlabel=r'Learning Rate ($\log_{10}$)',
)
vis.add_cbar(
    fig, ax, data['val.macro avg.f1-score'],
    label='$F_1$-score', vmin=0, vmax=1,
)
vis.add_grid(ax)
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn_x=lr,y=npl1,h=f1.pdf',
    bbox_inches='tight'
)

### N-layer FFNNs

In [ ]:
(
    data
    .groupby(['ffnn.num_layers'])
    [['val.macro avg.recall',  'val.macro avg.precision', 'val.macro avg.f1-score']]
    .agg(['max'])
#     .T
    .round(3)
)

#### x=hpar y=recall, all hyperparameters

In [ ]:
%%script false --no-raise-error
for nlayers in [1, 2, 3]:
    aspect=2
    subset_hpars = [
        hpar for hpar in hyperpars
        if data.loc[data['ffnn.num_layers'] == nlayers, hpar].nunique() > 1
    ]
    print([data.loc[data['ffnn.num_layers'] == nlayers, hpar].nunique() for hpar in hyperpars])
    print(subset_hpars)
    g = sns.catplot(
        data=vis.add_jitter(
            data[data['ffnn.num_layers'] == nlayers], subset_hpars
        ).melt( 
            id_vars=['Recall $>$ 0.7'], 
            value_vars=subset_hpars,
        ), 
        x="value", 
        col="variable",
        col_wrap=2,
        hue="Recall $>$ 0.7",
        kind="violin",
        sharex=False,

        dodge=True,
        inner='stick',
        linewidth=1,
        split=True,

    #     size=2.5,
    #     jitter=0.25,
    #     alpha=0.5,
        height=WIDTH*0.5 / aspect,
        legend=False,
        aspect=aspect,
    )

    for ax in g.axes.flat:
        hpar = ax.get_title().split(' = ')[1]
        ax.set_title(f'{rename_hpars[hpar]}')
    plt.tight_layout()

#### x=p,y=r,h=lr,c=nlayers

In [ ]:
# Three clusters in top right of precision-recall plots 
# -> caused by learning rate which needs to be _just_ right.
fig, axs = plt.subplots(1, 3, figsize=(WIDTH, 2))
for ax, nlayers in zip(axs, [1, 2, 3]):
    sns.scatterplot(
        data=vis.add_jitter(data[data['ffnn.num_layers'] == nlayers], hyperpars, amount=0),
        x='val.macro avg.precision',
        y='val.macro avg.recall',
        hue='nn.learning_rate.log10',
        hue_norm=(
            data['nn.learning_rate.log10'].min(), 
            data['nn.learning_rate.log10'].max()
        ),
        legend=False,
        palette='Spectral',
        **scatterplot_kwargs,
        ax=ax,
    )
    vis.rename_ax(ax)
    ax.set(
        title=f"{nlayers}-layer FFNNs",
        xlim=(-0.05, 1.05),
        ylim=(-0.05, 1.05),
        xticks=np.arange(0, 1.01, 0.25),
        yticks=np.arange(0, 1.01, 0.25),
    )
    if nlayers != 1:
        ax.set(
            ylabel=None,
            yticklabels=[],
        )
    vis.add_grid(ax)
vis.add_cbar(
    fig, axs[-1], data['nn.learning_rate.log10'],
    label=vis.rename_hpars['nn.learning_rate.log10']
)
plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,x=p,y=r,h=lr,c=nlayers.pdf',
    bbox_inches='tight',
)
plt.show()

#### x=lr y=f1,h=lr,c=nlayers

In [ ]:
%%script false --no-raise-error # This cell is unused
# Three clusters in top right of precision-recall plots 
# -> caused by learning rate which needs to be _just_ right.
fig, axs = plt.subplots(1, 3, figsize=(BIG_WIDTH, 2.5))
for ax, nlayers in zip(axs, [1, 2, 3]):
    sns.scatterplot(
        data=vis.add_jitter(
            data[data['ffnn.num_layers'] == nlayers], 
            hyperpars, 
            amount=0
        ),
        x='nn.learning_rate.log10',
        y='val.macro avg.f1-score',
        hue='nn.learning_rate.log10',
        edgecolor='#0005',
        legend=False,
        s=10,
        palette='Spectral',
        ax=ax,
    )
    ax.set(
        title=f"{nlayers}-layer FFNNs",
#         xlim=(-0.05, 1.05),
        ylim=(-0.05, 1.05),
        xticks=np.arange(-6, -0.95, 1)
    )
    if nlayers != 1:
        ax.set(
            ylabel=None,
            yticklabels=[],
        )

    vis.add_grid(ax)
#     vis.fmt_legend(ax)
    vis.rename_ax(ax)
vis.add_cbar(
    fig, axs[-1], data['nn.learning_rate.log10'],
    label=vis.rename_hpars['nn.learning_rate.log10']
)

plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,x=lr,y=f1,h=lr,c=nlayers.pdf'
)
plt.show()

#### x=lr,y=f1,h=npl-1,c=nlayers

In [ ]:
%%script false --no-raise-error # This cell is unused
# The last layer of the FFNNs have a big impact on the overall performance
# Generally the best-performing FFNNs have many nodes in the last layer, 
# and reducing the number of nodes reduces the performance
fig, axs = plt.subplots(1, 3, figsize=(8, 3))
for ax, nlayers in zip(axs, [1, 2, 3]):
    sns.scatterplot(
        data=vis.add_jitter(data[data['ffnn.num_layers'] == nlayers], hyperpars, amount=0),
        x='nn.learning_rate.log10',
        y='val.macro avg.f1-score',
        hue='ffnn.nodes_per_layer.-1.log10',
        edgecolor='#0005',
#         legend=False,
        s=10,
        palette='Spectral',
        ax=ax,
    )
    ax.set(
        title=f"{nlayers}-layer FFNNs",
#         xlim=(-0.05, 1.05),
        ylim=(-0.05, 1.05),
        xticks=np.arange(-6, -1, 1)
    )
    vis.add_grid(ax)
    vis.fmt_legend(ax)
plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,x=lr,y=f1,h=npl-1,c=nlayers.pdf'
)
plt.show()

#### x=lr,y=npl-1,h=f1,c=nlayers

In [ ]:
%%script false --no-raise-error # This cell is unused
# nodes in last layer vs learning rate and F_1
fig, axs = plt.subplots(1, 3, figsize=(WIDTH, 2))
for ax, nlayers in zip(axs, [1, 2, 3]):
    sns.scatterplot(
        data=vis.add_jitter(data[data['ffnn.num_layers'] == nlayers], hyperpars, amount=0.05),
        x='nn.learning_rate.log10',
        hue='val.macro avg.f1-score',
        y='ffnn.nodes_per_layer.-1.log10',
        edgecolor='#0005',
        legend=False,
        hue_norm=(0, 1),
        s=10,
        palette='Spectral',
        ax=ax,
    )
#     if nlayers == 1:
#         vis.fmt_legend(ax, title='$F_1$')
        
    ax.set(
        title=f"{nlayers}-layer FFNNs",
        xlim=(-6.5, -0.5),
        xticks=np.arange(-6, -.5, 1),
        ylim=(0.5, 2.75),
        yticks=np.arange(0.5, 2.75, 0.5),
    )
    vis.rename_ax(ax)
    vis.add_grid(ax)
    if nlayers != 1:
        ax.set(
            ylabel=None,
            yticklabels=[],
        )
vis.add_cbar(
    fig, axs[-1], data['val.macro avg.f1-score'],
    label=vis.rename_hpars['val.macro avg.f1-score'],
    vmin=0,vmax=1,
)

plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,x=lr,y=npl-1,h=f1,c=nlayers.pdf',
    bbox_inches='tight',
)
plt.show()

#### x=npl1,y=npl2,h=f1,c=nlayers

In [ ]:
%%script false --no-raise-error # This cell is unused
# More nodes generally means they do better
fig, axs = plt.subplots(2, 2, figsize=(WIDTH, 5.5))

sns.stripplot(
    data=vis.add_jitter(
        data[data['ffnn.num_layers'] == 1], 
        hyperpars, 
        amount=0.05
    ),
    x='ffnn.nodes_per_layer.1.log10',
    hue='val.macro avg.f1-score',
    hue_norm=(0, 1),
    s=3,
    edgecolor='#0005',
    linewidth=.25,
    legend=False,
    jitter=.2,
    palette='Spectral',
    ax=axs[0, 0]
)
# axs[0, 0].set_ylabel(vis.rename_hpars['val.macro avg.f1-score'])
# vis.fmt_legend(axs[0, 0], title='$F_1$', loc='upper left')

sns.scatterplot(
    data=vis.add_jitter(
        data[data['ffnn.num_layers'] == 2], 
        hyperpars, 
    ),
    x='ffnn.nodes_per_layer.1.log10',
    y='ffnn.nodes_per_layer.2.log10',
    hue='val.macro avg.f1-score',
    hue_norm=(0, 1),
    s=10,
    edgecolor='#0005',
    legend=False,
    palette='Spectral',
    ax=axs[0, 1]
)

sns.scatterplot(
    data=vis.add_jitter(
        data[data['ffnn.num_layers'] == 3], 
        hyperpars, 
    ),
    x='ffnn.nodes_per_layer.1.log10',
    y='ffnn.nodes_per_layer.2.log10',
    hue='val.macro avg.f1-score',
    hue_norm=(0, 1),
    s=10,
    edgecolor='#0005',
    legend=False,
    palette='Spectral',
    ax=axs[1, 0]
)

sns.scatterplot(
    data=vis.add_jitter(
        data[data['ffnn.num_layers'] == 3], 
        hyperpars, 
    ),
    y='ffnn.nodes_per_layer.2.log10',
    x='ffnn.nodes_per_layer.3.log10',
    hue='val.macro avg.f1-score',
    hue_norm=(0, 1),
    s=10,
    edgecolor='#0005',
    legend=False,
    palette='Spectral',
    ax=axs[1, 1]
)

for ax, nlayers in zip(axs.flatten(), [1, 2, 3, 3]):
    ax.set(
        title=f'{nlayers}-layer FFNNs',
        xlabel=rename_hpars.get(ax.get_xlabel(), None),
        ylabel=rename_hpars.get(ax.get_ylabel(), None),
    )
    vis.add_grid(ax)

    
for ax in axs[:, 1]:
    vis.add_cbar(
        fig, ax, data['val.macro avg.f1-score'],
        label=vis.rename_hpars['val.macro avg.f1-score'],
        vmin=0, vmax=1,
    )    

plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,x=npl1,y=npl2,h=f1,c=nlayers.pdf'
)
plt.show()

#### x=npl1,y=npl2,z=npl3,h=f1

In [ ]:
%%script false --no-raise-error # This cell is unused
# 3D plot of npl1 vs npl2 vs npl3 for 3-layer FFNNs
fig = plt.figure(dpi=200, figsize=(WIDTH*0.5, WIDTH*0.5))
ax = fig.add_subplot(projection='3d')

data_ = vis.add_jitter(
    data.loc[data['ffnn.num_layers'] == 3],
    ['ffnn.nodes_per_layer.1.log10', 'ffnn.nodes_per_layer.2.log10', 'ffnn.nodes_per_layer.3.log10'],
    amount=0.075
)
norm = plt.Normalize(0, 1)
ax.scatter(
    data_['ffnn.nodes_per_layer.1.log10'],
    data_['ffnn.nodes_per_layer.2.log10'],
    data_['ffnn.nodes_per_layer.3.log10'],
    c=data_['val.macro avg.f1-score'],
    s=1,
    alpha=1,
    cmap='Spectral',
    norm=norm,
)
ax.set(
    xlabel='\#Nodes (layer 1, $\log_{10}$)',
    ylabel='\#Nodes (layer 2, $\log_{10}$)',
    zlabel='\#Nodes (layer 3, $\log_{10}$)',
    title=vis.rename_hpars['val.macro avg.f1-score'] + ' against nodes per layer',
    xlim=(0.4, 2.75),
    ylim=(0.4, 2.75),
    zlim=(0.4, 2.75),
)

ax.view_init(35, -45)

plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,x=npl1,y=npl2,z=npl3,h=f1.pdf',
    bbox_inches='tight',
)

In [ ]:
%%script false --no-raise-error # Not for publication -- too big
# x=dropout
# y=nodes per layer
# cols = nlayers
# rows = layer index

fig, axs = plt.subplots(3, 3, figsize=(10,10))

for i, nlayers in enumerate([1,2,3]):
    for j, layer_idx in enumerate([1,2,3]):
        ax = axs[i, j]
        if layer_idx > nlayers:
            ax.axis('off')
            continue

        data_ = data[data['ffnn.num_layers'] == nlayers]
        
        sns.scatterplot(
            data=vis.add_jitter(data_, hyperpars, amount=0.05),
            x='ffnn.dropout_rate',
            hue='val.macro avg.f1-score',
            y=f'ffnn.nodes_per_layer.{layer_idx}.log10',
            s=20,
            legend=False,
            edgecolor='#0005',
            palette='Spectral',
            ax=ax,
            hue_norm=(0, 1),
#             vmin=0,
#             vmax=1,
        )
        
        ax.set(
            title=f'{nlayers}-layer FFNNs, layer {layer_idx}',
            xlabel=None if nlayers != 3 else ax.get_xlabel(),
#             xticks=np.round(ax.get_xticks(), 2),
#             xticklabels=[] if nlayers != 3 else np.round(ax.get_xticks(), 2),
            ylabel=None if layer_idx != 1 else ax.get_ylabel(),
#             yticks=np.round(ax.get_yticks(), 2),
#             yticklabels=[] if layer_idx != 1 else np.round(ax.get_yticks(), 2),
        )
#         vis.fmt_legend(ax)
        vis.add_grid(ax)
        vis.add_cbar(
            fig, ax, data['val.macro avg.f1-score'], 
            label='$F_1$', vmin=0, vmax=1,
        )
plt.tight_layout()

#### x=dropout,y=npl-1,h=npl1,c=nlayers

In [ ]:
%%script false --no-raise-error # This cell is unused
fig, axs = plt.subplots(1,3, figsize=(WIDTH,WIDTH*0.5))
for ax, nlayers in zip(axs, [1,2,3]):
    sns.scatterplot(
        data=vis.add_jitter(data[data['ffnn.num_layers'] == nlayers], hyperpars, amount=0.05),
        x='ffnn.dropout_rate',
        hue='val.macro avg.f1-score',
        y='ffnn.nodes_per_layer.-1.log10',
#         s=20,
        **scatterplot_kwargs,
#         edgecolor='black',
    #     legend=False,
        palette='Spectral',
        ax=ax,
    )
    vis.fmt_legend(ax)
    vis.add_grid(ax)
    ax.set(
        title=f'{nlayers}-layer FFNNs\n'
              f'(coloured by {rename_hpars["val.macro avg.f1-score"]})',
    )
plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51fffnn,x=dropout,y=npl-1,h=npl1,c=nlayers.pdf',
    bbox_inches='tight',
)
plt.show()

#### x=dropout,y=f1,h=l2,c=nlayers

In [ ]:
%%script false --no-raise-error # This cell is unused
fig, axs = plt.subplots(1,3, figsize=(BIG_WIDTH, 2.5))
for ax, nlayers in zip(axs, [1,2,3]):
    sns.scatterplot(
        data=vis.add_jitter(data[data['ffnn.num_layers'] == nlayers], hyperpars, amount=0.05),
        x='ffnn.dropout_rate',
        y='val.macro avg.f1-score',
        hue='ffnn.l2_coefficient.log10',
        hue_norm=(data['ffnn.l2_coefficient.log10'].min(),data['ffnn.l2_coefficient.log10'].max()),
        edgecolor='#0005',
        s=10,
        palette='Spectral',
        ax=ax,
        legend=False
    )
#     vis.fmt_legend(ax)
    vis.add_grid(ax)
    vis.rename_ax(ax)
    
    ax.set(
        title=f'{nlayers}-layer FFNNs',
        xticks=np.arange(0, 0.51, 0.1),
        xlim=(-0.01, 0.51),
        ylabel=None if nlayers != 1 else ax.get_ylabel(),
        ylim=(-0.05, 1.05),
        yticks=np.arange(0, 1.05, 0.2),
        yticklabels=[] if nlayers != 1 else np.round(np.arange(0, 1.05, 0.2), 1),
    )
vis.add_cbar(
    fig, ax, data['ffnn.l2_coefficient.log10'],
    label=vis.rename_hpars['ffnn.l2_coefficient.log10'],
    vmin=data['ffnn.l2_coefficient.log10'].min(),
    vmax=data['ffnn.l2_coefficient.log10'].max(),
)    

plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,x=dropout,y=f1,h=l2,c=nlayers.pdf',
    bbox_inches='tight',
)
plt.show()

#### x=dropout,y=npl-1,h=npl1,c=nlayers

In [ ]:
%%script false --no-raise-error # This cell is unused
fig, axs = plt.subplots(1,3, figsize=(BIG_WIDTH, 2.5))
for ax, nlayers in zip(axs, [1,2,3]):
    sns.scatterplot(
        data=vis.add_jitter(data[data['ffnn.num_layers'] == nlayers], hyperpars, amount=0.05),
        x='ffnn.dropout_rate',
        hue='val.macro avg.f1-score',
        y='ffnn.nodes_per_layer.-1.log10',
        hue_norm=(0, 1),
        s=10,
        edgecolor='#0005',
        legend=False,
        palette='Spectral',
        ax=ax,
    )
#     vis.fmt_legend(ax)
    vis.add_grid(ax)
    vis.rename_ax(ax),
    ax.set(
        title=f'{nlayers}-layer FFNNs',
        xticks=np.arange(0, 0.51, 0.1),
        xlim=(-0.01, 0.51),
        
        ylabel=None if nlayers != 1 else ax.get_ylabel(),
        ylim=(.5, 2.8),
        yticks=np.arange(.5, 2.8, 0.5),
        yticklabels=[] if nlayers != 1 else np.round(np.arange(.5, 2.8, 0.5), 1),
    )
    
vis.add_cbar(
    fig, ax, data['val.macro avg.f1-score'], vmin=0, vmax=1,
    label=vis.rename_hpars['val.macro avg.f1-score'],
)
plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51fffnn,x=dropout,y=npl-1,h=npl1,c=nlayers.pdf',
    bbox_inches='tight',
)
plt.show()

#### loss ratios

In [ ]:
%%script false --no-raise-error # This cell is unused
## loss ratios
fig, axs = plt.subplots(1, 2, figsize=(8, 4))
# for ax, nlayers in zip(axs, [1, 2, 3]):
sns.scatterplot(
    data=data,
#     data=data[data['ffnn.num_layers'] == nlayers],
    x=np.log10(data['trn.loss']),
    y=np.log10(data['val.loss']),
    edgecolor='#0005',
    hue='ffnn.num_layers',
#         style='ffnn.num_layers',
    palette='tab10',
    s=10,
    ax=axs[0],
)
axs[0].plot(
    [-1.25, 0.5],
    [-1.25, 0.5],
    c='#0005',
    alpha=0.5,
    linewidth=1,
)
vis.add_grid(axs[0])
vis.fmt_legend(axs[0], title='\#Layers')
axs[0].set(
    xlabel='Training Loss ($\\log_{10}$)',
    ylabel='Validation Loss ($\\log_{10}$)',
)

sns.scatterplot(
    data=data,
#     data=data[data['ffnn.num_layers'] == nlayers],
    y=(data['ratio.loss']),
    x=np.log10(data['trn.loss']),
    edgecolor='#0005',
    hue='ffnn.num_layers',
#         style='ffnn.num_layers',
    palette='tab10',
    s=10,
    ax=axs[1],
)
vis.add_grid(axs[1])
vis.fmt_legend(axs[1], title='\#Layers')
axs[1].plot(
    [-1.25, 1.0],
    [1, 1],
    c='#0005',
    alpha=0.5,
    linewidth=1,
)
axs[1].set(
    ylabel='$\\frac{Training}{Validation}$ Loss',
    xlabel='Training Loss ($\\log_{10}$)',
)
plt.suptitle(
    f'Training and Validation loss'
)

plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,x=trnloss,y=valloss,h=nlayers.pdf',
    bbox_inches='tight',
)

# ratio = training / validation
# low ratio -> training lower than validation -> overfitting (maybe)

#### 1-layer FFNNs

In [ ]:
data1 = data[data['ffnn.num_layers'] == 1]

In [ ]:
%%script false --no-raise-error # This cell is unused
# Precision-recall plots for 1-layer FFNNs with hue for each hyperparameter
fig, axs = plt.subplots(2, 3, figsize=(BIG_WIDTH, 4))
axs = axs.flatten()
subset_hyperpars = [
    'ffnn.num_layers',
    'nn.learning_rate.log10',
    'nn.batch_size.log10',
    'ffnn.nodes_per_layer.1.log10',
#     'ffnn.nodes_per_layer.2.log10',
#     'ffnn.nodes_per_layer.3.log10',
#     'ffnn.nodes_per_layer.-1.log10',
    'ffnn.l2_coefficient.log10',
    'ffnn.dropout_rate',
]
for hpar, ax in zip(subset_hyperpars, axs):
    sns.scatterplot(
        data=vis.add_jitter(data1, subset_hyperpars, amount=0.1),
        x='val.macro avg.precision',
        y='val.macro avg.recall',
        hue=hpar,
        s=5,
        edgecolor='#0005',
        legend=data1[hpar].nunique() < 10,
        palette='Spectral' if data1[hpar].nunique() > 10 else 'tab10',
        ax=ax,
    )
    if data1[hpar].nunique() > 10:
        vis.add_cbar(
            fig, ax, data1[hpar],
#             label=rename_hpars[hpar],
        )
    else:
        vis.fmt_legend(ax, title=rename_hpars[hpar])
    ax.set(
        title=rename_hpars[hpar],
        xlabel=rename_hpars['val.macro avg.precision'],
        ylabel=rename_hpars['val.macro avg.recall'],
        xlim=(-0.05, 1.05),
        ylim=(-0.05, 1.05),
        xticks=np.arange(0, 1.1, 0.25),
        yticks=np.arange(0, 1.1, 0.25),
    )
    vis.add_grid(ax)

for i, ax in enumerate(axs):
    if i % 3 != 0:
        ax.set(
            ylabel=None,
            yticklabels=[]
        )
    if i // 3 != (len(axs)-1)//3:
        ax.set(
            xlabel=None,
            xticklabels=[]
        )
    
for i in range(len(subset_hyperpars), len(axs)):
    axs[i].axis('off')

plt.subplots_adjust(
    wspace=0.25,
    hspace=0.35,
)
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,x=p,y=r,h=hpar,c=hpar,nlayers=1.pdf',
    bbox_inches='tight',
)
plt.show()

In [ ]:
# Hyperparameter-F1 plots
fig, axs = plt.subplots(2, 3, figsize=(WIDTH, 4))
axs = axs.flatten()
subset_hyperpars = [
    'ffnn.num_layers',
    'nn.learning_rate.log10',
    'nn.batch_size.log10',
    'ffnn.nodes_per_layer.1.log10',
#     'ffnn.nodes_per_layer.2.log10',
#     'ffnn.nodes_per_layer.3.log10',
#     'ffnn.nodes_per_layer.-1.log10',
    'ffnn.l2_coefficient.log10',
    'ffnn.dropout_rate',
]

for hpar, ax in zip(subset_hyperpars, axs):
    print(hpar)
    if data1[hpar].nunique() < 10:
        print(hpar, data1[hpar].nunique())
        sns.stripplot(
            data=data1,
            y='val.macro avg.f1-score',
            x=hpar,
            jitter=0.25,
            **stripplot_kwargs,
            ax=ax,
        )
    else:
        sns.scatterplot(
            data=data1,
            y='val.macro avg.f1-score',
            x=hpar,
            **scatterplot_kwargs,
            ax=ax,
        )
    ax.set(
        xlabel=rename_hpars[hpar],
        ylabel=rename_hpars['val.macro avg.f1-score'],
        ylim=(-0.05, 1.05),
        yticks=np.arange(0, 1.1, 0.25),
    )
    if data1[hpar].nunique() < 10:
        ax.set_xticklabels(data1[hpar].unique())

    vis.add_grid(ax)

for i, ax in enumerate(axs):
    if i % 3 != 0:
        ax.set(
            ylabel=None,
            yticklabels=[]
        )    
for i in range(len(subset_hyperpars), len(axs)):
    axs[i].axis('off')

plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,x=hpar,y=f1,c=hpar,nlayers=1.pdf',
    bbox_inches='tight',
)
plt.show()

#### 2-layer FFNNs

In [ ]:
data2 = data[data['ffnn.num_layers'] == 2]

In [ ]:
%%script false --no-raise-error # This cell is unused
# Precision-recall plots for 1-layer FFNNs with hue for each hyperparameter
fig, axs = plt.subplots(3, 3, figsize=(WIDTH, 5.5))
axs = axs.flatten()
subset_hyperpars = [
    'ffnn.num_layers',
    'nn.learning_rate.log10',
    'nn.batch_size.log10',
    'ffnn.nodes_per_layer.1.log10',
    'ffnn.nodes_per_layer.2.log10',
#     'ffnn.nodes_per_layer.3.log10',
#     'ffnn.nodes_per_layer.-1.log10',
    'ffnn.l2_coefficient.log10',
    'ffnn.dropout_rate',
]
for hpar, ax in zip(subset_hyperpars, axs):
    sns.scatterplot(
        data=vis.add_jitter(data2, subset_hyperpars, amount=0.1),
        x='val.macro avg.precision',
        y='val.macro avg.recall',
        hue=hpar,
        s=5,
        edgecolor='#0005',
        legend=data2[hpar].nunique() < 10,
        palette='Spectral' if data2[hpar].nunique() > 10 else 'tab10',
        ax=ax,
    )
    if data2[hpar].nunique() > 10:
        vis.add_cbar(
            fig, ax, data2[hpar],
#             label=rename_hpars[hpar],
        )
    else:
        vis.fmt_legend(ax, title=rename_hpars[hpar])
    ax.set(
        title=rename_hpars[hpar],
        xlabel=rename_hpars['val.macro avg.precision'],
        ylabel=rename_hpars['val.macro avg.recall'],
        xlim=(-0.05, 1.05),
        ylim=(-0.05, 1.05),
        xticks=np.arange(0, 1.1, 0.25),
        yticks=np.arange(0, 1.1, 0.25),
    )
    vis.add_grid(ax)

for i, ax in enumerate(axs):
    if i % 3 != 0:
        ax.set(
            ylabel=None,
            yticklabels=[]
        )
    if i + 3 < len(subset_hyperpars):
        ax.set(
            xlabel=None,
            xticklabels=[]
        )
    
for i in range(len(subset_hyperpars), len(axs)):
    axs[i].axis('off')

# plt.tight_layout()

plt.subplots_adjust(
    wspace=0.25,
    hspace=0.35,
)

plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,x=p,y=r,h=hpar,c=hpar,nlayers=2.pdf',
    bbox_inches='tight',
)
plt.show()

In [ ]:
# Hyperparameter-F1 plots
fig, axs = plt.subplots(3, 3, figsize=(WIDTH, 5.5))
axs = axs.flatten()
subset_hyperpars = [
    'ffnn.num_layers',
    'nn.learning_rate.log10',
    'nn.batch_size.log10',
    'ffnn.nodes_per_layer.1.log10',
    'ffnn.nodes_per_layer.2.log10',
#     'ffnn.nodes_per_layer.3.log10',
#     'ffnn.nodes_per_layer.-1.log10',
    'ffnn.l2_coefficient.log10',
    'ffnn.dropout_rate',
]
for hpar, ax in zip(subset_hyperpars, axs):
    if data2[hpar].nunique() < 10:
        sns.stripplot(
            data=data2,
            y='val.macro avg.f1-score',
            jitter=0.25,
            **stripplot_kwargs,
#             s=2.5,
#             linewidth=.5,
#             edgecolor='#0005',
            ax=ax,
        )
    else:
        sns.scatterplot(
            data=data2,
            y='val.macro avg.f1-score',
            x=hpar,
#             s=5,
#             edgecolor='#0005',
            **scatterplot_kwargs,
            ax=ax,
        )
    ax.set(
        xlabel=rename_hpars[hpar],
        ylabel=rename_hpars['val.macro avg.f1-score'],
        ylim=(-0.05, 1.05),
        yticks=np.arange(0, 1.1, 0.25),
    )
#     if data2[hpar].nunique() < 10:
#         ax.set_xticklabels([2])
    if data2[hpar].nunique() < 10:
        ax.set_xticklabels(data2[hpar].unique())


    vis.add_grid(ax)

for i, ax in enumerate(axs):
    if i % 3 != 0:
        ax.set(
            ylabel=None,
            yticklabels=[]
        )    
for i in range(len(subset_hyperpars), len(axs)):
    axs[i].axis('off')

# plt.subplots_adjust(
#     wspace=0.25,
#     hspace=0.35,
# )

plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,x=hpar,y=f1,c=hpar,nlayers=2.pdf',
    bbox_inches='tight',
)
plt.show()

#### 3-layer FFNNs

In [ ]:
data3 = data[data['ffnn.num_layers'] == 3]

In [ ]:
%%script false --no-raise-error # This cell is unused
# Precision-recall plots for 1-layer FFNNs with hue for each hyperparameter
fig, axs = plt.subplots(3, 3, figsize=(BIG_WIDTH, 5.5))
axs = axs.flatten()
subset_hyperpars = [
    'ffnn.num_layers',
    'nn.learning_rate.log10',
    'nn.batch_size.log10',
    'ffnn.nodes_per_layer.1.log10',
    'ffnn.nodes_per_layer.2.log10',
    'ffnn.nodes_per_layer.3.log10',
#     'ffnn.nodes_per_layer.-1.log10',
    'ffnn.l2_coefficient.log10',
    'ffnn.dropout_rate',
]
for hpar, ax in zip(subset_hyperpars, axs):
    sns.scatterplot(
        data=vis.add_jitter(data3, subset_hyperpars, amount=0.1),
        x='val.macro avg.precision',
        y='val.macro avg.recall',
        hue=hpar,
        **scatterplot_kwargs,
#         s=5,
#         edgecolor='#0005',
        legend=data3[hpar].nunique() < 10,
        palette='Spectral' if data3[hpar].nunique() > 10 else 'tab10',
        ax=ax,
    )
    if data3[hpar].nunique() > 10:
        vis.add_cbar(
            fig, ax, data3[hpar],
#             label=rename_hpars[hpar],
        )
    else:
        vis.fmt_legend(ax, title=rename_hpars[hpar])
    ax.set(
        title=rename_hpars[hpar],
        xlabel=rename_hpars['val.macro avg.precision'],
        ylabel=rename_hpars['val.macro avg.recall'],
        xlim=(-0.05, 1.05),
        ylim=(-0.05, 1.05),
        xticks=np.arange(0, 1.1, 0.25),
        yticks=np.arange(0, 1.1, 0.25),
    )
    vis.add_grid(ax)

for i, ax in enumerate(axs):
    if i % 3 != 0:
        ax.set(
            ylabel=None,
            yticklabels=[]
        )
    if i + 3 < len(subset_hyperpars):
        ax.set(
            xlabel=None,
            xticklabels=[]
        )
    
for i in range(len(subset_hyperpars), len(axs)):
    axs[i].axis('off')

plt.subplots_adjust(
    wspace=0.25,
    hspace=0.35,
)
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,x=p,y=r,h=hpar,c=hpar,nlayers=3.pdf',
    bbox_inches='tight',
)
plt.show()

In [ ]:
# Hyperparameter-F1 plots
fig, axs = plt.subplots(3, 3, figsize=(WIDTH, 5.5))
axs = axs.flatten()
subset_hyperpars = [
    'ffnn.num_layers',
    'nn.learning_rate.log10',
    'nn.batch_size.log10',
    'ffnn.nodes_per_layer.1.log10',
    'ffnn.nodes_per_layer.2.log10',
    'ffnn.nodes_per_layer.3.log10',
#     'ffnn.nodes_per_layer.-1.log10',
    'ffnn.l2_coefficient.log10',
    'ffnn.dropout_rate',
]
for hpar, ax in zip(subset_hyperpars, axs):
    if data3[hpar].nunique() < 10:
        sns.stripplot(
            data=data3,
            y='val.macro avg.f1-score',
            jitter=0.25,
            **stripplot_kwargs,
#             s=2.5,
#             linewidth=.5,
#             edgecolor='#0005',
            ax=ax,
        )
    else:
        sns.scatterplot(
            data=data3,
            y='val.macro avg.f1-score',
            x=hpar,
            **scatterplot_kwargs,
#             s=5,
#             edgecolor='#0005',
            ax=ax,
        )
    ax.set(
        xlabel=rename_hpars[hpar],
        ylabel=rename_hpars['val.macro avg.f1-score'],
        ylim=(-0.05, 1.05),
        yticks=np.arange(0, 1.1, 0.25),
    )
#     if data3[hpar].nunique() < 10:
#         ax.set_xticklabels([2])
    if data3[hpar].nunique() < 10:
        ax.set_xticklabels(data3[hpar].unique())


    vis.add_grid(ax)

for i, ax in enumerate(axs):
    if i % 3 != 0:
        ax.set(
            ylabel=None,
            yticklabels=[]
        )    
for i in range(len(subset_hyperpars), len(axs)):
    axs[i].axis('off')

# plt.tight_layout()

# plt.subplots_adjust(
#     wspace=0.25,
#     hspace=0.35,
# )

plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51ffnn,x=hpar,y=f1,c=hpar,nlayers=3.pdf',
    bbox_inches='tight',
)
plt.show()

#### x=dropout,y=npl-1,h=npl1,c=nlayers

In [ ]:
%%script false --no-raise-error # This cell is unused
fig, axs = plt.subplots(1,3, figsize=(WIDTH,WIDTH*.5))
for ax, nlayers in zip(axs, [1,2,3]):
    sns.scatterplot(
        data=vis.add_jitter(data[data['ffnn.num_layers'] == nlayers], hyperpars, amount=0.05),
        x='ffnn.dropout_rate',
        hue='val.macro avg.f1-score',
        y='ffnn.nodes_per_layer.-1.log10',
        s=20,
        edgecolor='black',
    #     legend=False,
        palette='Spectral',
        ax=ax,
    )
    vis.fmt_legend(ax)
    vis.add_grid(ax)
    ax.set(
        title=f'{nlayers}-layer FFNNs\n'
              f'(coloured by {rename_hpars["val.macro avg.f1-score"]})',
    )
plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_51fffnn,x=dropout,y=npl-1,h=npl1,c=nlayers.pdf',
    bbox_inches='tight',
)
plt.show()

# In depth HFFNN plots



In [ ]:
maj_min_hyperpars = [
    'hffnn.{}.ffnn.dropout_rate',
    'hffnn.{}.ffnn.l2_coefficient.log10',
    'hffnn.{}.nn.batch_size.log10',
    'hffnn.{}.nn.learning_rate.log10',
    'hffnn.{}.nn.epochs',
    'hffnn.{}.ffnn.num_layers',
    'hffnn.{}.ffnn.nodes_per_layer.1.log10',
    'hffnn.{}.ffnn.nodes_per_layer.2.log10',
    'hffnn.{}.ffnn.nodes_per_layer.3.log10',
    'hffnn.{}.ffnn.nodes_per_layer.-1.log10',
]

hyperpars = [
    hpar.format(subtype) 
    for hpar in maj_min_hyperpars
    for subtype in ['majority', 'minority'] 
]

data = df[
    (df['preprocessing.num_gesture_classes'] == '51') 
    & (df['model_type'] == 'HFFNN')
    # some of the HFFNNs didn't get their hyperparameters recorded. Exclude them
    & (df['hffnn.majority.ffnn.dropout_rate'].notna())
]
data['$F_1 >$ 0.5'] = data['val.macro avg.f1-score'] > 0.5

#### x=nlayers,y=f1,h=nlayers,c=majmin

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(WIDTH, 5))
axs = axs.flatten()
kwargs_arr = [
    dict(
        y='val.macro avg.f1-score',
        x='Maj. \#Layers',
        hue='Min. \#Layers',
        ax=axs[0],
    ),
    dict(
        y='val.macro avg.f1-score',
        x='Min. \#Layers',
        hue='Maj. \#Layers',
        ax=axs[1]
    )
]
data_ = data.assign(**{
    'Maj. \#Layers': lambda ddf: ddf['hffnn.majority.ffnn.num_layers'].astype(int),
    'Min. \#Layers': lambda ddf: ddf['hffnn.minority.ffnn.num_layers'].astype(int),
})
for kwargs in kwargs_arr:
    sns.stripplot(
        **kwargs,
        data=data_,
#         s=2.5,
        palette='tab10' if data_[kwargs['hue']].nunique() < 10 else 'Spectral',
#         linewidth=0.5,
        jitter=0.25,
#         edgecolor='#0005',
        **stripplot_kwargs,
    )
    vis.add_grid(kwargs['ax'])
    vis.fmt_legend(
        kwargs['ax'], 
        title=kwargs['hue'],
        loc='best',
    )
    kwargs['ax'].set(
        xlabel=kwargs['x'],
        ylabel=kwargs['y'],
        ylim=(-0.05, 1.25),
        yticks=np.arange(0, 1.1, 0.25),
        yticklabels=np.arange(0, 1.1, 0.25),
    )
    vis.rename_ax(kwargs['ax'])
        

    
for i, hue in enumerate(['Min. \#Layers', 'Maj. \#Layers']):
    sns.scatterplot(
        data=data_,
        x='val.macro avg.precision',
        y='val.macro avg.recall',
        hue=hue,
        ax=axs[2+i],
        **scatterplot_kwargs,
        palette='tab10',
    )

    vis.rename_ax(axs[2+i])
    vis.add_grid(axs[2+i])
    axs[2+i].set(
        xlim=(0, 1),
        ylim=(0, 1),
    )

plt.tight_layout()

plt.savefig(
    '../../report/src/imgs/graphs/05_51hffnn,x=nlayers,y=f1,h=nlayers,c=majmin.pdf',
    bbox_inches='tight',
)

In [ ]:

fig, axs = plt.subplots(
    2,2,
    figsize=(WIDTH, 5)
)
axs = axs.flatten()
axs[-1].axis('off')
jittered = vis.add_jitter(data, hyperpars, amount=0.05, clamp=False)

sns.scatterplot(
    data=jittered,
    hue='hffnn.minority.ffnn.nodes_per_layer.-1.log10',
    x='hffnn.majority.ffnn.nodes_per_layer.-1.log10',
    y='val.macro avg.f1-score',
    palette='Spectral',
    s=10,
    edgecolor='#0005',
    ax=axs[0],
    legend=False,
    
)

sns.scatterplot(
    data=jittered,
    hue='hffnn.majority.ffnn.nodes_per_layer.-1.log10',
    x='hffnn.minority.ffnn.nodes_per_layer.-1.log10',
    y='val.macro avg.f1-score',
    palette='Spectral',
    s=10,
    edgecolor='#0005',
    ax=axs[1],
    legend=False,
)

sns.scatterplot(
    data=jittered.sort_values('val.macro avg.f1-score'),
    y='hffnn.minority.ffnn.nodes_per_layer.-1.log10',
    x='hffnn.majority.ffnn.nodes_per_layer.-1.log10',
    hue='val.macro avg.f1-score',
    hue_norm=(0, 1),
    palette='Spectral',
    s=10,
    edgecolor='#0005',
    legend=False,
    ax=axs[2]
)

axs[0].set(
    xlabel=rename_hpars['hffnn.majority.ffnn.nodes_per_layer.-1.log10'],
    ylabel=rename_hpars['val.macro avg.f1-score'],
    ylim=(-0.05, 1.05),
    yticks=np.arange(0, 1.1, 0.2),
)
axs[1].set(
    xlabel=rename_hpars['hffnn.minority.ffnn.nodes_per_layer.-1.log10'],
    ylabel=rename_hpars['val.macro avg.f1-score'],
#     ylabel=None,
    yticks=np.arange(0, 1.1, 0.2),
#     yticklabels=[],
    ylim=(-0.05, 1.05),
)

# for ax in axs[[0, 1]]:
vis.add_cbar(
    fig, axs[1], data['hffnn.majority.ffnn.nodes_per_layer.1.log10'],
    label=rename_hpars['ffnn.nodes_per_layer.-1.log10'],
    vmin=np.log10(4),
    vmax=np.log10(512),
)

axs[2].set(
    xlim=(0.45, 2.8),
    ylim=(0.45, 2.8),
    xlabel=rename_hpars['hffnn.majority.ffnn.nodes_per_layer.-1.log10'],
    ylabel=rename_hpars['hffnn.minority.ffnn.nodes_per_layer.-1.log10'],
)

vis.add_cbar(
    fig, axs[2], data['val.macro avg.f1-score'],
    label=rename_hpars['val.macro avg.f1-score'],
    vmin=0,
    vmax=1,
)

vis.add_grid(axs[0])
vis.add_grid(axs[1])
vis.add_grid(axs[2])

plt.tight_layout()

plt.savefig(
    '../../report/src/imgs/graphs/05_51hffnn,x=npl-1,y=f1,h=majmin,c=majmin.pdf'
)

#### x=lr,y=f1,h=lr,c=majmin

In [ ]:
fig, axs = plt.subplots(
    2, 2,
    figsize=(WIDTH, 5.5)
)
axs = axs.flatten()
axs[-1].axis('off')

sns.scatterplot(
    data=vis.add_jitter(data, hyperpars, amount=0.025),
    x='hffnn.majority.nn.learning_rate.log10',
    hue='hffnn.minority.nn.learning_rate.log10',
    y='val.macro avg.f1-score',
    palette='Spectral',
    s=10,
    edgecolor='#0005',
    ax=axs[0],
    legend=False,
)

axs[0].set(
    xlabel=rename_hpars['hffnn.majority.nn.learning_rate.log10'],
    ylabel=rename_hpars['val.macro avg.f1-score'],
    xticks=np.arange(-6, -0.95, 1),
    ylim=(0, 1),
    title='Majority LR against $F_1$\n'
          '(colour is the Min. LR)'
)

sns.scatterplot(
    data=vis.add_jitter(data, hyperpars, amount=0.025),
    hue='hffnn.majority.nn.learning_rate.log10',
    x='hffnn.minority.nn.learning_rate.log10',
    y='val.macro avg.f1-score',
    palette='Spectral',
    s=10,
    edgecolor='#0005',
    ax=axs[1],
    legend=False,
)
vis.add_cbar(
    fig, axs[1], data['hffnn.majority.nn.learning_rate.log10'],
    label=rename_hpars['nn.learning_rate.log10'],
    vmin=-6,
    vmax=-1,
)

axs[1].set(
    xlabel=rename_hpars['hffnn.minority.nn.learning_rate.log10'],
    xticks=np.arange(-6, -0.95, 1),
    ylabel=vis.rename_hpars['val.macro avg.f1-score'],
    yticks=axs[0].get_yticks(),
#     yticklabels=[],
    title='Minority LR against $F_1$\n'
          '(colour is the Maj. LR)'
)
vis.add_grid(axs[0])
vis.add_grid(axs[1])

sns.scatterplot(
    data=vis.add_jitter(data, hyperpars, amount=0.025),
    x='hffnn.majority.nn.learning_rate.log10',
    y='hffnn.minority.nn.learning_rate.log10',
    hue='val.macro avg.f1-score',
    hue_norm=(0, 1),
    palette='Spectral',
    s=10,
    edgecolor='#0005',
    ax=axs[2],
    legend=False,
)
vis.add_cbar(
    fig, axs[2], data['val.macro avg.f1-score'],
    vmin=0, vmax=1, label='$F_1$'
)

vis.rename_ax(axs[2])
plt.tight_layout()

plt.savefig(
    '../../report/src/imgs/graphs/05_51hffnn,x=lr,y=f1,h=lr,c=majmin.pdf'
)

#### x=p,y=r,h=hpar,col=hpar

In [ ]:
%%script false --no-raise-error # This cell is unused
fig, axs = plt.subplots(5, 4, figsize=(16, 20), dpi=300)

for i, ax in enumerate(axs.flat):
    if i >= len(hyperpars):
        ax.axis('off')
        continue
    sns.scatterplot(
        data=data,
        x='val.macro avg.precision',
        y='val.macro avg.recall',
        hue=hyperpars[i],
        edgecolor='#0005',
        s=10,
        ax=ax,
#         legend=False,
        palette='Spectral' if data[hyperpars[i]].nunique() > 5 else 'tab10',
    )
    ax.set(
        xlabel=None,
        ylabel=None,
        title=vis.rename_hpars[hyperpars[i]],
        xlim=(0, 1),
        ylim=(0, 1),
    )
    vis.fmt_legend(ax)
    vis.add_grid(ax)

plt.tight_layout()
plt.show()

#### x=majhpar,y=minhpar,h=f1

In [ ]:
%%script false --no-raise-error # This cell is unused
fig, axs = plt.subplots(5, 4, figsize=(10, 10), dpi=300)

for i, ax in enumerate(axs.flat):
    if i >= len(hyperpars):
        ax.axis('off')
        continue
    kwargs = dict(
        hue=(
            hyperpars[i].replace('maj', 'min')
            if 'maj' in hyperpars[i] else
            hyperpars[i].replace('min', 'maj')
        ),
        x=hyperpars[i],
        y='val.macro avg.f1-score',
    )
    discrete = data[kwargs['x']].nunique() < 5
    jitter_amount = 0.025 if not discrete else 0.15
    jitter_hpars = hyperpars if not discrete else [kwargs['x']]
    sns.scatterplot(
        data=vis.add_jitter(data, jitter_hpars, amount=jitter_amount, clamp=not discrete),
        **kwargs,
        edgecolor='#0005',
        s=5,
        hue_norm=(
            data[kwargs['hue']].min(),
            data[kwargs['hue']].max(),
        ),
        ax=ax,
        legend=discrete,
        palette='Spectral' if not discrete else 'tab10',
    )
    ax.set(
        xlabel=vis.rename_hpars[kwargs['x']],
        ylabel=vis.rename_hpars[kwargs['y']],
        ylim=(-0.05, 1.05),
        yticks=np.arange(0, 1.01, 0.2),
        yticklabels=np.round(np.arange(0, 1.01, 0.2), 1),
#         title=rename_hpars[kwargs['hue']],
    )
#     vis.fmt_legend(ax)
    vis.add_grid(ax)
    if discrete:
        vis.fmt_legend(ax)
    else:

        vis.add_cbar(
            fig, ax, 
            data[kwargs['hue']],
            label=vis.rename_hpars[kwargs['hue']],
            vmin=data[kwargs['hue']].min(),
            vmax=data[kwargs['hue']].max(),
            cmap='Spectral',
        )

plt.tight_layout()
plt.show()

In [ ]:
%%script false --no-raise-error # This cell is unused
fig, axs = plt.subplots(3, 3, figsize=(10, 10), dpi=200)

hpar = 'nn.learning_rate.log10'

for i, majlayers in enumerate([1, 2, 3]):
    for j, minlayers in enumerate([1, 2, 3]):
        ax = axs[i, j]
        sns.scatterplot(
            data=vis.add_jitter(data[
                (data['hffnn.majority.ffnn.num_layers'] == majlayers)
                & (data['hffnn.minority.ffnn.num_layers'] == minlayers)
            ], hyperpars, amount=0.05),
            x=f'hffnn.majority.{hpar}',
            y=f'hffnn.minority.{hpar}',
            hue='val.macro avg.f1-score',
            ax=ax,
            edgecolor='#0005',
            s=20,
            palette='Spectral',
            legend=False,
        )
        ax.set(
            title=f'MajLayers: {majlayers} MinLayers: {minlayers}',
        )
plt.tight_layout()

In [ ]:
%%script false --no-raise-error # This cell is unused
sns.scatterplot(
    data=vis.add_jitter(
        data,
        ['hffnn.majority.ffnn.num_layers', 'hffnn.minority.ffnn.num_layers'],
        amount=0.1,
        clamp=False,
    ),
    x='hffnn.majority.ffnn.num_layers',
    y='hffnn.minority.ffnn.num_layers',
    hue='val.macro avg.f1-score',
    palette='Spectral',
    legend=False,
    s=10,
)

#### all HFFNN triplets

In [ ]:
%%script false --no-raise-error
# This plot is broken because it's not needed
for i, maj_min_hyperpar in enumerate(maj_min_hyperpars):
    maj_hpar = maj_min_hyperpar.format('majority')
    min_hpar = maj_min_hyperpar.format('minority')

    fig, axs = plt.subplots(
        2, 2,
        figsize=(WIDTH, 5.5)
    )
    axs = axs.flatten()
    axs[-1].axis('off')
    
    kwargs_arr = [
        dict(x=maj_hpar, y='val.macro avg.f1-score', hue=min_hpar),
        dict(x=min_hpar, y='val.macro avg.f1-score', hue=maj_hpar),
        dict(x=maj_hpar, y=min_hpar, hue='val.macro avg.f1-score'),
    ]

    for j, kwargs in enumerate(kwargs_arr):
        continuous = data[kwargs['x']].nunique() > 5
        
        jitter_hpars = (
            [kwargs['x']]
            if kwargs['y'] == 'val.macro avg.f1-score' else
            [kwargs['x'], kwargs['y']]
        )
        jitter_amount = 0.025 if continuous else 0.2

        sns.scatterplot(
            **kwargs,
            data=vis.add_jitter(
                data, 
                jitter_hpars, 
                amount=jitter_amount,
                clamp=continuous,
            ),
            palette='Spectral' if continuous else ['tab:blue', 'tab:orange', 'tab:green'],
            s=10,
            edgecolor='#0005',
            ax=axs[j],
            legend=continuous,
        )
#         print(kwargs, continuous)
    
    axs[0].set(
        xlabel=rename_hpars[maj_hpar],
        ylabel=rename_hpars['val.macro avg.f1-score'],
        ylim=(0, 1),
        title=f'{vis.rename_hpars[maj_hpar]} vs $F_1$\n'
              f'(colour is {vis.rename_hpars[min_hpar]})'
    )
    if data[maj_hpar].nunique() > 5:
        vis.add_cbar(
            fig, axs[1], data[maj_hpar],
            label=rename_hpars[maj_min_hyperpar[9:]],
        )
    else:
        vis.fmt_legend(axs[1], title=rename_hpars[maj_min_hyperpar[9:]])

    axs[1].set(
        xlabel=rename_hpars[min_hpar],
#         xticks=np.arange(-6, -0.95, 1),
        ylabel=vis.rename_hpars['val.macro avg.f1-score'],
        yticks=axs[0].get_yticks(),
#         yticklabels=[],
        title=f'{vis.rename_hpars[min_hpar]} vs $F_1$\n'
              f'(colour is {vis.rename_hpars[maj_hpar]})'
    )
    vis.add_grid(axs[0])
    vis.add_grid(axs[1])
    vis.add_grid(axs[2])

    vis.add_cbar(
        fig, axs[2], data['val.macro avg.f1-score'],
        vmin=0, vmax=1, 
        label=vis.rename_hpars['val.macro avg.f1-score']
    )

    vis.rename_ax(axs[2])
    plt.tight_layout()

#     plt.savefig(
#         f'../../report/src/imgs/graphs/05_51hffnn,triplets={maj_min_hyperpar[9:]}.pdf'
#     )
    plt.show()


### Distribution plots

In [ ]:
%%script false --no-raise-error # This cell is unused
maj_min = (
    'majority',
    'minority',
)
for submodel in maj_min:
    hyperpars = (
        f'hffnn.{submodel}.ffnn.dropout_rate',
        f'hffnn.{submodel}.ffnn.l2_coefficient.log10',
        f'hffnn.{submodel}.nn.batch_size.log10',
        f'hffnn.{submodel}.nn.learning_rate.log10',
        f'hffnn.{submodel}.ffnn.nodes_per_layer.1',
        f'hffnn.{submodel}.ffnn.nodes_per_layer.2',
        f'hffnn.{submodel}.ffnn.nodes_per_layer.3',
    )
    df[[
        f'hffnn.{submodel}.ffnn.l2_coefficient.log10',
        f'hffnn.{submodel}.nn.batch_size.log10',
        f'hffnn.{submodel}.nn.learning_rate.log10',
    ]] = np.log10(df[[
        f'hffnn.{submodel}.ffnn.l2_coefficient',
        f'hffnn.{submodel}.nn.batch_size',
        f'hffnn.{submodel}.nn.learning_rate',
    ]])
    xlabels = (
        f'Dropout Rate\n({submodel.title()} Classifier)',
        f'L2 Coefficient ($\log_{{10}}$)\n({submodel.title()} Classifier)',
        f'Batch Size ($\log_{{10}}$)\n({submodel.title()} Classifier)',
        f'Learning Rate ($\log_{{10}}$)\n({submodel.title()} Classifier)',
        f'Nodes in Layer 1\n({submodel.title()} Classifier)',
        f'Nodes in Layer 2\n({submodel.title()} Classifier)',
        f'Nodes in Layer 3\n({submodel.title()} Classifier)',
    )

    data = df[
        (df['preprocessing.num_gesture_classes'] == '51') &
        (df['model_type'] == 'HFFNN')
    ]

    fig, axs = plt.subplots(
        3, 3,
        figsize=(WIDTH, WIDTH)
    )
    print(axs.shape)
    for i, ax in enumerate(axs.flatten()):
        if len(hyperpars) <= i:
            ax.axis('off')
            continue

        sns.scatterplot(
            data=data,
            x=hyperpars[i],
            y='val.macro avg.f1-score',
            s=10,
            alpha=0.5,
            ax=ax,
            color='tab:orange',
            edgecolor=None,
        )
        title_xlabel = xlabels[i].replace(" ($\log_{10}$)", "")
        ax.set(
            title=f'$F_1$ score vs {title_xlabel}',
            xlabel=xlabels[i],
            ylabel='$F_1$ score',
            ylim=(-0.1, 1.1),
        )
    plt.tight_layout()
#     plt.savefig(
#         f'../../report/src/imgs/graphs/05_in_depth_hffnn_{submodel}_hpars.pdf',
#         bbox_inches='tight',
#     )
    plt.show()

## In depth CuSUM plots

In [ ]:
nclasses_list = ('5', '50', '51')
for nclasses in nclasses_list:
    data = df[
        (df['preprocessing.num_gesture_classes'] == nclasses)
        & (df['model_type'] == 'CuSUM')
    ]

    fig, axs = plt.subplots(1, 2, figsize=(WIDTH, WIDTH*.5))
    p_min = max(-0.05, data['val.macro avg.precision'].min() / 1.10)
    p_max = min( 1.05, data['val.macro avg.precision'].max() * 1.10)
    r_min = max(-0.05, data['val.macro avg.recall'].min()    / 1.10)
    r_max = min( 1.05, data['val.macro avg.recall'].max()    * 1.10)

    sns.scatterplot(
        data=data,
        y='val.macro avg.recall',
        x='val.macro avg.precision',
        hue='cusum.thresh',
        **scatterplot_kwargs,
#         alpha=0.75,
#         s=10,
        ax=axs[0],
        palette=palette,
        legend=False,
#         edgecolor=None
    )

    sns.stripplot(
        data=data,
        x='cusum.thresh',
        y='val.macro avg.f1-score',
        **stripplot_kwargs,
#         alpha=0.5,
#         s=2.5,
        ax=axs[1],
        jitter=0.25,
        color=model_colours['CuSUM'],
        native_scale=True,
    )

    p_range = p_max - p_min
    r_range = r_max - r_min
    axs[0].set(
        title=f"Precision vs recall\n{nclasses}-class CuSUM models",
        xlabel='Precision',
        ylabel='Recall',
        xlim=(p_min - p_range*0.025, p_max + p_range*0.025),
        ylim=(r_min - r_range*0.025, r_max + r_range*0.025),
    )

#     axs[0].legend().set_title('Threshold')
    vis.add_cbar(
        fig, axs[0], data=data['cusum.thresh'],
        label='Threshold'
    )

    axs[1].set(
        title=f'Threshold vs $F_1$ score\n{nclasses}-class CuSUM models',
        xlabel='CuSUM Threshold',
        ylabel='$F_1$ score',
    )
    for ax in axs.flat:
        vis.add_grid(ax)
    plt.tight_layout()
    plt.savefig(
        f'../../report/src/imgs/graphs/05_hpar_analysis_cusum_classes{nclasses}.pdf',
        bbox_inches='tight',
    )
    plt.show()

## In depth SVM plots

- Correlation with the training time per observation
- Correlation with the precision/recall/f1
- clusters in recall-precision space

In [ ]:
# Get some summary stats
(
    df.loc[(df['model_type'] == 'SVM')]
    .groupby(['preprocessing.num_gesture_classes'], dropna=False)
    [['val.macro avg.f1-score', 'val.macro avg.precision', 'val.macro avg.recall']]
    .agg(['count', 'median', 'mean', 'min', 'max', 'std'])
    .T
    .round(3)
)

In [ ]:
n_classes = ('5', '50', '51')
for num_gesture_classes in n_classes:
    data = df[
        (df['preprocessing.num_gesture_classes'] == num_gesture_classes)
        & (df['model_type'] == 'SVM')
    ]

    data['svm.c.log10'] = np.log10(data['svm.c'])
    data['svm.class_weight'] = data['svm.class_weight'].fillna('unbalanced')

    data = data.rename(columns={
        'svm.c.log10': '$\log_{10}(C)$', 
        'svm.class_weight': 'Class Weight'
    })

    fig, axs = plt.subplots(1, 2, figsize=(WIDTH, 0.5*WIDTH))
    axs = axs.flatten()
    p_min = data['val.macro avg.precision'].min() / 1.05
    p_max = data['val.macro avg.precision'].max() * 1.05
    r_min = data['val.macro avg.recall'].min()    / 1.05
    r_max = data['val.macro avg.recall'].max()    * 1.05

    sns.scatterplot(
        data=data,
        y='val.macro avg.recall',
        x='val.macro avg.precision',
        hue='$\log_{10}(C)$',
        style='Class Weight',
#         alpha=0.8,
        **scatterplot_kwargs,
#         s=10,
        ax=axs[0],
        palette=palette,
        legend=False,
#         edgecolor='#0005',
    )

    axs[0].set(
        title=f"Precision vs Recall"
        f"\n({num_gesture_classes}-class SVM)",
        xlabel='Precision',
        ylabel='Recall',
    )

    sns.scatterplot(
        data=data,
        x='$\log_{10}(C)$',
        y='val.macro avg.f1-score',
        style='Class Weight',
        **scatterplot_kwargs,
#         alpha=0.8,
#         s=10,
        ax=axs[1],
        color='tab:purple',
#         legend=False,
#         edgecolor='#0005',
    )

    axs[1].set(
        title=f"$F_1$-score vs C\n"
        f"({num_gesture_classes}-class SVM)",
        xlabel='C ($\log_{10}$)',
        ylabel='$F_1$-score',
    )
    
#     move_legend(axs[0], axs[-1])
    vis.add_cbar(
        fig, axs[0],
        data['$\log_{10}(C)$'],
        label='$\log_{10}(C)$',
    )
#     axs[-2].axis('off')
    vis.fmt_legend(axs[1])
    vis.add_grid(axs[0])
    vis.add_grid(axs[1])
    plt.tight_layout()
    plt.savefig(
        f'../../report/src/imgs/graphs/05_in_depth_svm_classes{num_gesture_classes}.pdf',
        bbox_inches='tight',
    )
    plt.show()

In [ ]:
nclasses_list = ('5', '50', '51')
for nclasses in nclasses_list:
    data = df[
        (df['preprocessing.num_gesture_classes'] == nclasses)
        & (df['model_type'] == 'SVM')
    ]

    data['svm.class_weight'] = data['svm.class_weight'].fillna('unbalanced')

    conf_mats = {}
    conf_mat_totals = {}
    precisions = {}
    recalls = {}
    f1_scores = {}

    hpar = 'svm.class_weight'

    assert len(data[hpar].unique()) < 10

    for i, row in data.iterrows():
        y_true, y_pred = get_npz_data_from_model('../' + row['model_dir'])
        cm = tf.math.confusion_matrix(
            y_true.flatten(), 
            y_pred.flatten()
        ).numpy()
        hpar_item = row[hpar]

        precision, recall, f1_score, _support = sklearn.metrics.precision_recall_fscore_support(
            y_true.flatten(), 
            y_pred.flatten(),
            zero_division=0,
        )


        if hpar_item in conf_mats:
            conf_mats[hpar_item] += cm.astype(float)
            conf_mat_totals[hpar_item] += 1.0
            precisions[hpar_item] += precision
            recalls[hpar_item] += recall
            f1_scores[hpar_item] += f1_score

        else:
            conf_mats[hpar_item] = cm.astype(float)
            conf_mat_totals[hpar_item] = 1.0
            precisions[hpar_item] = precision
            recalls[hpar_item] = recall
            f1_scores[hpar_item] = f1_score


    fig, axs = plt.subplots(
        1, len(conf_mats), 
        figsize=(WIDTH, WIDTH/len(conf_mats))
    )


    for i, (hpar_item, conf_mat) in enumerate(conf_mats.items()):
    #     conf_mat[-1, -1] = 0
        conf_mat /= conf_mat_totals[hpar_item]
        vis.conf_mat(conf_mat, ax=axs[i])
        axs[i].set_title(
            f'{hpar_item.title()} SVMs'
        )

    plt.tight_layout()

    plt.savefig(
        f'../../report/src/imgs/graphs/05_in_depth_svm_conf_mats_unbalanced_classes{nclasses}.pdf',
        bbox_inches='tight',
    )
    plt.show()
#     fig, axs = plt.subplots(
#         1, 2,
#         figsize=(WIDTH, WIDTH*.2),
#     )
#     axs = axs.flatten()
#     for i, (hpar_item, conf_mat) in enumerate(conf_mats.items()):
#         axs[i].set_aspect(3)
#         vis.precision_recall_f1(
#             precision=precisions[hpar_item] / conf_mat_totals[hpar_item],
#             recall=recalls[hpar_item] / conf_mat_totals[hpar_item],
#             f1=f1_scores[hpar_item] / conf_mat_totals[hpar_item],
#             ax=axs[i],
#         )
#         axs[i].set_title(hpar_item.title() + " SVMs")
#     plt.tight_layout()

#     plt.savefig(
#         f'../../report/src/imgs/graphs/05_in_depth_svm_prf1_plots_unbalanced_classes{nclasses}.pdf',
#         bbox_inches='tight',
#     )
#     plt.show()


In [ ]:
data = df[
    (df['preprocessing.num_gesture_classes'] == '51')
    & (df['model_type'] == 'SVM')
]

data['svm.c.log10'] = np.log10(data['svm.c'])
data['svm.class_weight'] = data['svm.class_weight'].fillna('unbalanced')

data = data.rename(columns={
    'svm.c.log10': '$\log_{10}(C)$', 
    'svm.class_weight': 'Class Weight'
})
fig, axs = plt.subplots(1, 2, figsize=(WIDTH, WIDTH*0.5))

sns.scatterplot(
    data=data,
    y='fit_time_per_obs',
    x='$\log_{10}(C)$',
#     s=10,
#     alpha=0.5,
#     hue='',
    style='Class Weight',
    color='tab:purple',
    **scatterplot_kwargs,
    palette=palette,
    ax=axs[0],
#     edgecolor=None,
)
sns.scatterplot(
    data=data,
    y='trn.pred_time_per_obs',
    x='$\log_{10}(C)$',
#     s=10,
    **scatterplot_kwargs,
#     alpha=0.5,
#     hue='',
    style='Class Weight',
    color='tab:purple',
    palette=palette,
    ax=axs[1],
#     edgecolor=None,
    legend=False
)

axs[0].set(
    title='Fit time vs C\n51-class SVM',
    ylabel='Fit time (s/obs.)',
    xlabel='$\log_{10}(C)$',
)
axs[1].set(
    title='Inference time vs C\n51-class SVM',
    ylabel='Inference time (s/obs.)',
    xlabel='$\log_{10}(C)$',
)

vis.add_grid(axs[0])
vis.add_grid(axs[1])
vis.fmt_legend(axs[0])

plt.tight_layout()

plt.savefig(
    '../../report/src/imgs/graphs/05_svm_hpars_vs_fit_time.pdf',
    bbox_inches='tight'
)

In [ ]:
%%script false --no-raise-error # This cell is unused
# SVM 51-class fitting time vs f1-score style=class weight hue=C
data = df[
    (df['preprocessing.num_gesture_classes'] == '51')
    & (df['model_type'] == 'SVM')
]

data['svm.c.log10'] = np.log10(data['svm.c'])
data['svm.class_weight'] = data['svm.class_weight'].fillna('unbalanced')

# fig, axs = plt.subplots(1, 2, figsize=(WIDTH, WIDTH*0.5))

sns.scatterplot(
    data=data,
    y='fit_time_per_obs',
    x='val.macro avg.f1-score',
    s=20,
#     alpha=0.5,
    hue='svm.c.log10',
    style='svm.class_weight',
#     color='tab:purple',
    palette='Spectral',
#     ax=axs[0],
    edgecolor='#000',
)
plt.show()

data[[
    'fit_time_per_obs',
    'val.macro avg.f1-score',
    'svm.c.log10',
    'svm.class_weight',
]].to_clipboard()

## In depth HMM plots

- Clusters in recall-precision space

In [ ]:
# Get some summary stats
hue_order = ['Spherical', 'Diagonal', 'Tied', 'Full']
(
    df.loc[(df['model_type'] == 'HMM')]
    .groupby(['preprocessing.num_gesture_classes', 'hmm.covariance_type'])
    [['val.macro avg.f1-score', 'val.macro avg.precision', 'val.macro avg.recall']]
    .agg(['count', 'median', 'mean', 'min', 'max', 'std'])
    .T
    .round(3)
)

In [ ]:
model_type = 'HMM'
color='tab:red'
n_classes = ('5', '50', '51',)
for num_gesture_classes in n_classes:
    data = df[
        (df['preprocessing.num_gesture_classes'] == num_gesture_classes)
        & (df['model_type'] == model_type)
    ]

    data['hmm.covariance_type'] = data['hmm.covariance_type'].replace({
        'spherical': 'Spherical',
        'diag': 'Diagonal',
        'full': 'Full',
        'tied': 'Tied',
    })

    fig, axs = plt.subplots(1, 2, figsize=(WIDTH, WIDTH*0.5))
    p_min = data['val.macro avg.precision'].min() # / 1.10
    p_max = data['val.macro avg.precision'].max() # * 1.10
    r_min = data['val.macro avg.recall'].min()    # / 1.10
    r_max = data['val.macro avg.recall'].max()    # * 1.10

#     precision_grid, recall_grid = np.meshgrid(
#         np.linspace(p_min, p_max, 50), 
#         np.linspace(r_min, r_max, 50),
#     )

#     f1_score = 2 * (precision_grid * recall_grid) / (precision_grid + recall_grid)

#     contours = axs[0].contour(
#         precision_grid,
#         recall_grid, 
#         f1_score,
#         levels=np.linspace(
#             f1_score.min(),
#             f1_score.max(),
#             10,
#         ), 
#         colors='#0005',
#         alpha=0.25
#     )
#     axs[0].clabel(contours, inline=True, fontsize=8, fmt='%.2f')

    sns.scatterplot(
        data=data,
        y='val.macro avg.recall',
        x='val.macro avg.precision',
#         hue='val.macro avg.f1-score',
        hue='hmm.covariance_type',
        hue_order=hue_order,
        alpha=0.5,
        s=10,
        ax=axs[0],
        palette=other_colours,
        edgecolor=None,
    )

    p_range = p_max - p_min
    r_range = r_max - r_min
    axs[0].set(
        title=f"Precision vs Recall"
        f"\n({num_gesture_classes}-class {model_type}s)",
        xlabel='Precision',
        ylabel='Recall',
#         xlim=(0, .05),
#         ylim=(0, 1),
        xlim=(p_min - p_range*0.025, p_max + p_range*0.025),
        ylim=(r_min - r_range*0.025, r_max + r_range*0.025),
    )
    axs[0].legend().set_title('Covariance')
    
    
    sns.stripplot(
        data=data,
        y='val.macro avg.f1-score',
        x='hmm.covariance_type',
        order=hue_order,
        size=3,
        alpha=0.5,
        ax=axs[1],
        palette=other_colours,
#         hue='val.macro avg.f1-score',
        hue='hmm.covariance_type',
        hue_order=hue_order,

    )
    axs[1].set(
        title=f"$F_1$-score vs Covariance Type"
        f"\n({num_gesture_classes}-class {model_type}s)",
        xlabel='Covariance Type',
        ylabel='$F_1$-score',
#         xlim=(p_min - p_range*0.025, p_max + p_range*0.025),
#         ylim=(-0.05, 1.05),
    )
    axs[0].legend().set_title('Covariance Type')
    plt.tight_layout()
    vis.add_grid(axs[0])
    vis.add_grid(axs[1])
    vis.fmt_legend(axs[0])
    plt.savefig(
        f'../../report/src/imgs/graphs/05_in_depth_hmm_{num_gesture_classes}_p_vs_r_covar_type.pdf',
        bbox_inches='tight',
    )
    plt.show()

In [ ]:
%%script false --no-raise-error # This cell is unused

sns.scatterplot(
    data=df.assign(**{
        'calc_f1-score': lambda ddf: 2 * (ddf['val.macro avg.precision'] * ddf['val.macro avg.precision']) / (ddf['val.macro avg.recall'] + ddf['val.macro avg.recall'])
    }),
    x='val.macro avg.f1-score',
    y='calc_f1-score',
    hue='preprocessing.num_gesture_classes',
    s=.5,
    alpha=0.5,
    edgecolor=None,
)

In [ ]:
nclasses_list = ('5', '50', '51')
for nclasses in nclasses_list:
    fig, ax = plt.subplots(1, 1, figsize=(WIDTH*0.5, WIDTH*0.5))

    data = df[
        (df['preprocessing.num_gesture_classes'] == nclasses)
        & (df['model_type'] == 'HMM')
    ]

    data['val.pred_time_per_obs.log10'] = np.log10(data['val.pred_time_per_obs'])
    data['trn.pred_time_per_obs.log10'] = np.log10(data['trn.pred_time_per_obs'])
    data['fit_time_per_obs.log10'] = np.log10(data['fit_time_per_obs'])

    data['hmm.covariance_type'] = data['hmm.covariance_type'].replace({
        'spherical': 'Spherical',
        'diag': 'Diagonal',
        'full': 'Full',
        'tied': 'Tied',
    })


    sns.scatterplot(
        data=data,
        x='fit_time_per_obs.log10',
        y='trn.pred_time_per_obs.log10',
        s=5,
        alpha=0.5,
        hue='hmm.covariance_type',
        hue_order=hue_order,
        ax=ax,
        palette=other_colours,
        edgecolor=None,
    )

#     ax.legend().set_title('Covariance Type')

    ax.set(
        title=f'Fitting time vs inference time\n({nclasses}-class HMMs)',
        xlabel=r'Fit time ($\log_{10}(s/obs)$)',
        ylabel='Inference time ($\log_{10}(s/obs)$)',
    )
    vis.add_grid(ax)
    vis.fmt_legend(ax)

    plt.tight_layout()
    plt.savefig(
        f'../../report/src/imgs/graphs/05_in_depth_hmm_inf_trn_time_classes{nclasses}.pdf',
        bbox_inches='tight'
    )

In [ ]:
nclasses_list = ('5', '50', '51')
for nclasses in nclasses_list:

    data = df[
        (df['preprocessing.num_gesture_classes'] == nclasses)
        & (df['model_type'] == 'HMM')
    ]
    data['hmm.covariance_type'] = data['hmm.covariance_type'].replace({
        'spherical': 'Spherical',
        'diag': 'Diagonal',
        'full': 'Full',
        'tied': 'Tied',
    })

    conf_mats = {}
    conf_mat_totals = {}
    precisions = {}
    recalls = {}
    f1_scores = {}
    y_preds_dict = {}
    y_true_dict = {}
    reports_dict = {}

    hpar = 'hmm.covariance_type'

    assert len(data[hpar].unique()) < 10

    for i, row in data.iterrows():
        y_true, y_pred = get_npz_data_from_model('../' + row['model_dir'])
        precision, recall, f1_score, _support = sklearn.metrics.precision_recall_fscore_support(
            y_true.flatten(), 
            y_pred.flatten(),
    #         average='macro',
            zero_division=0,
        )

        cm = tf.math.confusion_matrix(
            y_true.flatten(), 
            y_pred.flatten()
        ).numpy()
        hpar_item = row[hpar]

        if hpar_item in conf_mats:
            conf_mats[hpar_item] += cm.astype(float)
            conf_mat_totals[hpar_item] += 1.0
            precisions[hpar_item] += precision
            recalls[hpar_item] += recall
            f1_scores[hpar_item] += f1_score
            y_preds_dict[hpar_item] = np.concatenate((y_preds_dict[hpar_item], y_pred.flatten()))
            y_true_dict[hpar_item] = np.concatenate((y_true_dict[hpar_item], y_true.flatten()))
        else:
            conf_mats[hpar_item] = cm.astype(float)
            conf_mat_totals[hpar_item] = 1.0
            precisions[hpar_item] = precision
            recalls[hpar_item] = recall
            f1_scores[hpar_item] = f1_score
            y_preds_dict[hpar_item] = np.copy(y_pred.flatten())
            y_true_dict[hpar_item] = np.copy(y_true.flatten())

    fig, axs = plt.subplots(
        2, 2,
        figsize=(WIDTH, WIDTH),
        squeeze=False,
    )
    axs = axs.flatten()
    hpar_items = ['Spherical', 'Diagonal', 'Tied', 'Full']
    for i, hpar_item in enumerate(hpar_items):
        conf_mat = conf_mats[hpar_item]
        vis.conf_mat(conf_mat, ax=axs[i])
    #     vis.conf_mat(conf_mat / conf_mat.max(), ax=axs[i], norm=None)
        axs[i].set_title(
            f'{hpar_item.title()} {nclasses}-class HMMs'
    #         f'(Mean of {int(conf_mat_totals[hpar_item])} confusion matrices)'
        )

    plt.tight_layout()

    plt.savefig(
        f'../../report/src/imgs/graphs/05_in_depth_hmm_conf_mats_cov_type_classes{nclasses}.pdf',
        bbox_inches='tight',
    )
    plt.show()

#     fig, axs = plt.subplots(
#         2, 2,
#         figsize=(WIDTH, WIDTH*.4),
#     )
#     axs = axs.flatten()
#     hpar_items = ['Spherical', 'Diagonal', 'Tied', 'Full']
#     for i, hpar_item in enumerate(hpar_items):
#         conf_mat = conf_mats[hpar_item]
#     #     axs[i].set_aspect(1)
#         vis.precision_recall_f1(
#             precision=precisions[hpar_item] / conf_mat_totals[hpar_item],
#             recall=recalls[hpar_item] / conf_mat_totals[hpar_item],
#             f1=f1_scores[hpar_item] / conf_mat_totals[hpar_item],
#             ax=axs[i],
#         )
#         axs[i].set_title(f'{hpar_item} {nclasses}-class HMMs')
#         if i % 2 != 0:
#             axs[i].set_yticks([])
#     plt.tight_layout()

#     plt.savefig(
#         f'../../report/src/imgs/graphs/05_in_depth_hmm_prf1_plots_conv_type_classes{nclasses}.pdf',
#         bbox_inches='tight',
#     )
#     plt.show()

## Inference times vs num classes

In [ ]:
%%script false --no-raise-error # This cell is unused
df['val.pred_time_per_obs'] = df['val.pred_time'] / df['val.num_observations']
df['val.pred_time_per_obs.log10'] = np.log10(df['val.pred_time_per_obs'].fillna(0))

fig, axs = plt.subplots(2, 2, figsize=(WIDTH, WIDTH))
axs = axs.flatten()
model_types = [m for m in model_colours.keys() if m != 'HFFNN']
for ax, model_type in zip(axs, sorted(model_types)):
    sns.stripplot(
        data=df[
            df['model_type'] == model_type
        ].sort_values('preprocessing.num_gesture_classes'),
        x='preprocessing.num_gesture_classes',
        y='val.pred_time_per_obs.log10',
        hue='model_type',
#         dodge=True,
        s=2,
        legend=False,
        hue_order=list(model_colours.keys()),
        alpha=0.5,
        jitter=0.3,
        ax=ax,
    )
    ax.set(
        ylim=(-6.1, -0.9),
        title=f'{model_type} inference time\nagainst \#classes',
        xlabel='Number of Classes',
        ylabel='Inference time $\log_{10}$(s/obs)',
    )
    vis.add_grid(ax)

plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_inf_time_vs_num_classes.pdf',
    bbox_inches='tight',
)

## Training times vs num classes

In [ ]:
%%script false --no-raise-error # This cell is unused
df['fit_time_per_obs.log10'] = np.log10(df['fit_time_per_obs'].fillna(0))

fig, axs = plt.subplots(2, 2, figsize=(WIDTH, WIDTH))
axs = axs.flatten()
model_types = [m for m in model_colours.keys() if m != 'HFFNN']
for ax, model_type in zip(axs, sorted(model_types)):
    sns.stripplot(
        data=df[
            df['model_type'] == model_type
        ].sort_values('preprocessing.num_gesture_classes'),
        x='preprocessing.num_gesture_classes',
        y='fit_time_per_obs.log10',
        hue='model_type',
#         dodge=True,
        s=2,
        legend=False,
        hue_order=list(model_colours.keys()),
        alpha=0.5,
        jitter=0.3,
        ax=ax,
    )
    ax.set(
#         ylim=(-3.5, -0.5),
        title=f'{model_type} fitting time\nagainst \#classes',
        xlabel='Number of Classes',
        ylabel='Fitting time $\log_{10}$(s/obs)',
    )
    vis.add_grid(ax)

plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_fit_time_vs_num_classes.pdf',
    bbox_inches='tight',
)

## Inference times vs $F_1$-score

In [ ]:
%%script false --no-raise-error # This cell is unused
df['val.pred_time_per_obs'] = df['val.pred_time'] / df['val.num_observations']
df['val.pred_time_per_obs.log10'] = np.log10(df['val.pred_time_per_obs'].fillna(0))

fig, ax = plt.subplots(1, 1, figsize=(WIDTH, WIDTH))
sns.scatterplot(
    data=df[
        (df['preprocessing.num_gesture_classes'] == '51')
    ],
    y='val.macro avg.f1-score',
    x='val.pred_time_per_obs.log10',
    hue='model_type',
    s=5,
    edgecolor='#000a',
#     legend=False,
    hue_order=list(model_colours.keys()),
#         alpha=0.5,
#         jitter=0.3,
    ax=ax,
)
ax.set(
    xlim=(-6.1, -0.9),
    ylim=(-0.05, 1.05)
)
vis.add_grid(ax)

plt.tight_layout()

## PCA Plots

### PCA decomposition only including the gesture classes

In [ ]:
(
    X_trn, X_val, y_trn, y_val, dt_trn, dt_val
) = common.read_and_split_from_npz("../gesture_data/trn_20_10.npz")

In [ ]:
%%script false --no-raise-error
from sklearn.manifold import TSNE

shape = (np.nonzero(y_trn != 50)[0].shape[0] * 5,)
mask = np.concatenate((
    np.random.choice(np.nonzero(y_trn == 50)[0], shape), 
    np.nonzero(y_trn != 50)[0]
))

# fig, axs = plt.subplots(1, 3, figsize=(WIDTH, WIDTH/3), dpi=200)
fig, axs = plt.subplots(1, 3, figsize=(30, 20), dpi=200)
for i, perplexity in enumerate([35, 45, 55]):
    print(f"PERPLEXITY: {perplexity}")
    X_embedded = TSNE(
        n_components=2,
        n_iter=1500,
        learning_rate='auto',
        init='random',
        perplexity=perplexity,
        verbose=True
    ).fit_transform(X_trn[mask].reshape((X_trn[mask].shape[0], 600)))

    hues = np.array([ "0$^\circ$", "45$^\circ$", "90$^\circ$", "135$^\circ$", "180$^\circ$", "Non-gesture"])
    styles = np.array([ "L5", "L4", "L3", "L2", "L1", "R1", "R2", "R3", "R4", "R5", "Non-gesture"])
#     fig, ax = plt.subplots(1, 1, figsize=(WIDTH, WIDTH), dpi=200)
    argsort = np.argsort(y_trn[mask])

    sns.scatterplot(
        x=X_embedded[:, 0][argsort],
        y=X_embedded[:, 1][argsort],
        hue=hues[(y_trn[mask][argsort] // 10)],
        style=styles[(y_trn[mask][argsort] % 10)],
        s=10,
        alpha=0.5,
        ax=axs.flatten()[i],
        legend=False,
        edgecolor=None,
    )
    axs.flatten()[i].set_title(f"Perplexity: {perplexity}")

In [ ]:

def move_legend(from_ax, to_ax, title=None):
    # Fetch the current legend
    handles, labels = from_ax.get_legend_handles_labels()
    title = title if title is not None else from_ax.legend().get_title()
    # Remove it from the ax
    from_ax.legend().remove()
    # Add it to the new ax with some styling
    to_ax.legend(
        handles=handles, 
        labels=labels,
        loc='center',
        fancybox=False, 
        edgecolor="black",
        title=title
#         linewidth=0.5,
    )
    # Remove the axis of the ax
    to_ax.axis('off')
    return from_ax, to_ax

In [ ]:
%%script false --no-raise-error # This cell is unused

print("Executing PCA")
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_reshaped = X_trn.reshape((X_trn.shape[0], 600))
X_tfrm = pca.fit_transform(X_reshaped)
print("PCA complete")

n = 11
markers = [
    "o", "X", (4, 0, 45), "P", (4, 0, 0), (4, 1, 0), "^", (4, 1, 45), "v",
]

# Now generate more from regular polygons of increasing order
s = 5
while len(markers) < n:
    a = 360 / (s + 1) / 2
    markers.extend([(s + 1, 1, a), (s + 1, 0, a), (s, 1, 0), (s, 0, 0)])
    s += 1

markers = np.array([m for m in markers[:n]])
# markers

In [ ]:
%%script false --no-raise-error # This cell is unused

fig, ax = plt.subplots(1, 1, figsize=(WIDTH, WIDTH*0.8), dpi=300)
colours = np.array([ "tab:blue", "tab:orange", "tab:green", "tab:red", "tab:purple", 'k'])
# markers = np.array(['o', 'x', 's', '+', 'D', 'p', '^', 'd', 'v', '*', 'x'])

# fig, ax = plt.subplots(1, 1, figsize=(WIDTH, WIDTH), dpi=300)
# mask = (y_trn == 50)
# axs[1].scatter(
#     X_tfrm[:, 0][mask],
#     X_tfrm[:, 1][mask],
#     color='#0005',
#     alpha=0.1,
#     s=5,
#     edgecolor='none',
# )
for gidx in range(0, 51):
    ax.scatter(
        X_tfrm[:, 0][y_trn == gidx],
        X_tfrm[:, 1][y_trn == gidx],
        color=colours[gidx // 10],
        marker=markers[gidx % 10],
        alpha=0.75 if gidx != 50 else 0.1,
        zorder=10 if gidx != 50 else 0,
        s=10 if gidx != 50 else 5,
        edgecolor='none',
    )
ax.set_title('PCA plot of the training data\n(class 50 in black)')

ax.set(
    xlabel='Principal Component 1',
    ylabel='Principal Component 2',
    xlim=(-1400, 3000),
    ylim=(-1300, 1600),
)

handles, labels = ax.get_legend_handles_labels()
# display_ = (0,1,2)

#Create custom artists
color_artists = [
    plt.Line2D((0,1),(0,0), color=colour, marker='o', linestyle='')
    for colour in colours[:-1]
]
color_labels = ['$0^{\circ}$','$45^{\circ}$','$90^{\circ}$','$135^{\circ}$','$180^{\circ}$']

marker_artists = [
    plt.Line2D((0,1),(0,0), color='k', fillstyle='none', marker=marker, linestyle='')
    for marker in markers[:-1]
]
marker_labels = [
    'Left little',
    'Left ring',
    'Left middle',
    'Left index',
    'Left thumb',
    'Right thumb',
    'Right index',
    'Right middle',
    'Right ring',
    'Right little',
]
#Create legend from custom artist/label lists
ax.legend(
    color_artists + marker_artists + [plt.Line2D((0,1),(0,0), color='k', marker='o', linestyle='')],
    color_labels + marker_labels + ['Class 50'],
    loc='center right'
)
# vis.fmt_legend(ax)

plt.tight_layout()

plt.savefig(
    '../../report/src/imgs/graphs/05_pca_plot.png',
    bbox_inches='tight'
)

### PCA plot showing just an interesting subset

In [ ]:
%%script false --no-raise-error # This cell is unused
pca = PCA(n_components=2)
X_reshaped = X_trn.reshape((X_trn.shape[0], 600))
X_tfrm = pca.fit_transform(X_reshaped)

In [ ]:
%%script false --no-raise-error # This cell is unused

colours = np.array([ "tab:blue", "tab:orange", "tab:green", "tab:red", "tab:purple"])

@interact(
    x_start=(-1500, 1500, 50),
    y_start=(-1500, 1500, 50),
    x_length=(-1500, 1500, 50),
    y_length=(-1500, 1500, 50),
)
def fn(x_start=-150, y_start=1000, x_length=500, y_length=500):
    x_finsh = x_start + x_length
    y_finsh = y_start + y_length

    fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=100)

    selection_mask = (
        (x_start <= X_tfrm[:, 0]) & (X_tfrm[:, 0] <= x_finsh) &
        (y_start <= X_tfrm[:, 1]) & (X_tfrm[:, 1] <= y_finsh)
    )
    X_subset = X_tfrm[selection_mask]
    y_subset = y_trn[selection_mask]
#     ax.scatter(
#         X_subset[:, 0],
#         X_subset[:, 1],
#         c='black',
#         alpha=0.1,
#     )
    
    y_mask = (y_trn == 50)
    ax.scatter(
        X_subset[:, 0][y_subset == 50],
        X_subset[:, 1][y_subset == 50],
        color='black',
        alpha=0.1,
        s=20,
        edgecolor='none',
    )
    ax.scatter(
        X_subset[:, 0][y_subset != 50],
        X_subset[:, 1][y_subset != 50],
        color=colours[(y_subset[y_subset != 50] // 10)],
        alpha=0.75,
#         s=5,
        edgecolor='none',
    )


### PCA plot that connects sequential datapoints

In [ ]:
%%script false --no-raise-error # This cell is unused

pca = PCA(n_components=2)
X_reshaped = X_trn.reshape((X_trn.shape[0], 600))
X_tfrm = pca.fit_transform(X_reshaped)

order = np.argsort(dt_trn)
X_tfrm = X_tfrm[order]

In [ ]:
%%script false --no-raise-error # This cell is unused


limit = 1000
@interact(start=(0, len(X_tfrm), 25), length=(0, len(X_tfrm), 50))
def fn(start=0, length=500):
    finsh = min(start + length, len(X_tfrm))
    fig, ax = plt.subplots(1, 1, figsize=(WIDTH, WIDTH), dpi=100)
    ax.plot(
        X_tfrm[:, 0][start:finsh],
        X_tfrm[:, 1][start:finsh],
        zorder=0,
        c='#0005',
        alpha=0.1,
    )
    sns.scatterplot(
        x=X_tfrm[:, 0][start:finsh],
        y=X_tfrm[:, 1][start:finsh],
        hue=dt_trn[order][start:finsh],
        legend=False,
        s=(10 + 90*(y_trn != 50)[order][start:finsh]),
#         edgecolor=np.where((y_trn != 50)[order][start:finsh], '#0005', 'none'),
        linewidth=.5,
        edgecolor=None,
    )
    
    idxs = np.nonzero(y_trn[order][start:finsh] != 50)[0]
    for idx in idxs:
        ax.text(
            X_tfrm[start:finsh][idx, 0],
            X_tfrm[start:finsh][idx, 1],
            y_trn[order][start:finsh][idx],
            va='center',
            ha='center',
        )
#     ax.set_xlim((
#         X_tfrm[:, 0].min() / 1.1,
#         X_tfrm[:, 0].max() * 1.1,
#     ))
#     ax.set_ylim((
#         X_tfrm[:, 1].min() / 1.1,
#         X_tfrm[:, 1].max() * 1.1,
#     ))

### PCA plot with ellipses

In [ ]:
%%script false --no-raise-error # This cell is unused

pca = PCA(n_components=2)
X_reshaped = X_trn.reshape((X_trn.shape[0], 600))
X_tfrm = pca.fit_transform(X_reshaped[y_trn != 50])
y_tfrm = y_trn[y_trn != 50]

In [ ]:
%%script false --no-raise-error # This cell is unused

# https://matplotlib.org/stable/gallery/statistics/confidence_ellipse.html
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    pearson = cov[0, 1] / np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensional dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse(
        (0, 0), 
        width=ell_radius_x * 2, 
        height=ell_radius_y * 2,
        facecolor=facecolor, 
        **kwargs,
    )

    # Calculating the standard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the standard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)
    
#     print(f"{scale_x=}, {scale_y=}\n{mean_x=}, {mean_y=}")

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    
    return ax.add_patch(ellipse)

In [ ]:
%%script false --no-raise-error # This cell is unused

fig, ax = plt.subplots(1, 1, figsize=(WIDTH, WIDTH))
# gidx = 14

hues = np.array([ "0$^\circ$", "45$^\circ$", "90$^\circ$", "135$^\circ$", "180$^\circ$" ])
styles = np.array([ "L5", "L4", "L3", "L2", "L1", "R1", "R2", "R3", "R4", "R5" ])
markers = ['o', 's', 'D', '^', 'v', '>', '<', 'p', 'H', '+']
colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple']

for gidx in range(50):
    ax.scatter(
        X_tfrm[y_tfrm == gidx, 0],
        X_tfrm[y_tfrm == gidx, 1],
        color=colors[gidx // 10],
        marker=markers[gidx % 10],
        s=5,
        alpha=0.5,
        label=gidx
    )
    confidence_ellipse(
        X_tfrm[y_tfrm == gidx, 0],
        X_tfrm[y_tfrm == gidx, 1],
        ax,
        n_std=2,
        edgecolor=colors[gidx // 10],
        alpha=0.5,
    )
plt.legend()

## Visualise mis-predictions

1. Load in a continuous dataset
2. Load in a classifier
3. Use the classifier to make predictions on the dataset
4. Visualise the mispredictions, but *with context*

### Load in a model for which to evaluate the mis-predictions

In [ ]:
%%script false --no-raise-error # This cell is unused

# Load in the dataset/classifier
(
    X_trn, X_val, y_trn, y_val, dt_trn, dt_val
) = common.read_and_split_from_npz("../gesture_data/trn_20_10.npz")

clf = models.load_tf('../src/saved_models/ffnn_2023-09-18T14:05:16.363404')
const = common.read_constants('../src/constants.yaml')
sensor_names = list(const['sensors'].values())

### Visualise True and Mispredicted gestures

Plot all the observations which have the ground truth being gesture 255 but the model is not predicting g255

In [ ]:
%%script false --no-raise-error # This cell is unused

y_pred = clf.predict(X_val)

for gidx in np.unique(y_val):
    if gidx == 50: continue
    pred_indxs = np.nonzero((y_val == 50) & (y_pred == gidx))[0]
    true_indxs = np.nonzero(y_val == gidx)[0]
    axs = vis.cmp_ts(
        X_val[true_indxs],
    )
    vis.cmp_ts(
        X_val[pred_indxs],
        color='tab:red',
        axs=axs,
    )

#     distances = np.abs(true_indxs[:, np.newaxis] - pred_indxs).min(axis=0)

    plt.suptitle(f'Model predicted {gidx}, ground truth: 50 \
                 \nGesture {gidx} in grey, mispredicted in red ({len(pred_indxs)} observations) \
                 \nindices: {pred_indxs}')
    plt.tight_layout()
#     plt.savefig(f'../src/notebooks/pred_{gidx:0>2}_truth_50.pdf', bbox_inches='tight')
    plt.show()
#     if gidx > 5:
    break


## Eval model on test set

In [ ]:
model_dir = '../src/saved_models/ffnn_2023-10-08T14:47:07.901649'
tst_data = np.load('../gesture_data/tst_20_10.npz')
y_tst = tst_data['y_tst']
X_tst = tst_data['X_tst']
dt_tst = tst_data['dt_tst']
clf = models.load_tf(model_dir)

In [ ]:
model_type = clf.config['model_type']
y_pred = clf.predict(X_tst)
clf_report_dict = sklearn.metrics.classification_report(
    y_tst.astype(int),
    y_pred.astype(int),
    output_dict=True,
    zero_division=0,
)
clf_report = pd.json_normalize(clf_report_dict)
# print(sklearn.metrics.classification_report(
#     y_tst.astype(int),
#     y_pred.astype(int),
#     output_dict=False,
#     zero_division=0,
# ))
                            
f1 = clf_report['macro avg.f1-score'].values[0]
precision = clf_report['macro avg.precision'].values[0]
recall = clf_report['macro avg.recall'].values[0]
# print("val.macro avg.f1-score", f1)
# print("val.macro avg.precision", precision)
# print("val.macro avg.recall", recall)
# print (clf_report['macro avg.f1-score'].values[0])

cm = tf.math.confusion_matrix(y_tst, y_pred, num_classes=51).numpy()
# cm[-1, -1] = 0
fig, axs = plt.subplots(2, 1, figsize=(WIDTH, WIDTH))
vis.conf_mat(
    cm,
    ax=axs[0],
)
axs[0].set_title(
    f'Confusion Matrix (Test set)\n'
    f'$F_1$={np.round(f1, 3)}, Precision={np.round(precision, 3)}, Recall={np.round(recall, 3)}'
)

axs[1].set_aspect(2.5)
vis.precision_recall_f1(
    report=clf_report_dict,
    ax=axs[1],
)

plt.tight_layout()

plt.savefig(
    '../../report/src/imgs/graphs/05_tst_set_conf_mat.pdf'
)

In [ ]:
ddf = pd.DataFrame()
ddf['Recall'] = {
    int(k): d['recall']
    for k, d in clf_report_dict.items()
    if type(d) is dict and k.isdigit()
}
ddf['Precision'] = {
    int(k): d['precision']
    for k, d in clf_report_dict.items()
    if type(d) is dict and k.isdigit()
}
ddf['$F_1$-score'] = {
    int(k): d['f1-score']
    for k, d in clf_report_dict.items()
    if type(d) is dict and k.isdigit()
}
ddf = ddf.reset_index()
ddf

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(WIDTH, 0.9*WIDTH))

hues = np.array([ "0$^\circ$", "45$^\circ$", "90$^\circ$", "135$^\circ$", "180$^\circ$", 'Non-Gesture'])
styles = np.array([ "L5", "L4", "L3", "L2", "L1", "R1", "R2", "R3", "R4", "R5", 'Non-Gesture'])

sns.scatterplot(
    data=ddf,
    x='Precision',
    y='Recall',
    hue=hues[ddf.index // 10],
    style=styles[ddf.index % 10],
    ax=ax,
    s=20,
    linewidth=1,
    edgecolor='#0005',
)

for i, row in ddf.iterrows():
    ax.text(
        row['Precision'],
        row['Recall'],
        str(int(row['index'])),
        ha='center',
        va='bottom',
    )
vis.add_grid(ax)
ax.set(
    xlim=(-0.05, 1.05),
    ylim=(-0.05, 1.05),
    xticks=np.arange(0, 1.01, 0.1),
    yticks=np.arange(0, 1.01, 0.1),
)

ax.set(
    title='Precision and Recall for all gestures (best model)'
)
plt.savefig(
    '../../report/src/imgs/graphs/05_p_r_best_model.pdf'
)

# Interactive plot to see data at a certain time

In [ ]:
%%script false --no-raise-error # This cell is unused

df_trn = read.read_data(
    '../gesture_data/train/', 
    constants_path='../src/constants.yaml',
)
vc = df_trn['gesture'].value_counts()
vc = vc[vc.index != 'gesture0255']
vc

In [ ]:
%%script false --no-raise-error # This cell is unused

df = read.read_data(
    '../gesture_data/train/', 
    constants_path='../src/constants.yaml',
)
df['gidx'] = df['gesture'].apply(lambda g: int(g[-4:]) if g != 'gesture0255' else 50)

@interact(dt='2022-10-08T20:23:46.665276000')
def fn(dt='2022-10-08T20:23:46.665276000'):
    try:
        dt = pd.to_datetime(dt)
    except Exception as e:
        print(e)
        return
#     fig, ax = plt.subplots(1, 1, figsize=(WIDTH, WIDTH))
    mask = df['datetime'].between(
        dt - pd.to_timedelta(1, 'second'),
        dt + pd.to_timedelta(1, 'second')
    )
    
    vis.cmp_ts(
        [df.loc[mask, sensor_names].values]
    )


# Plot a CSV file + predictions

In [ ]:
# %%script false --no-raise-error # This cell is unused

csv_path = '../gesture_data/saved_from_cli_2023-10-08_tqbfjotld.csv'
# model_dir = '../src/saved_models/ffnn_2023-10-08T14:14:07.933452'
model_dir = '../src/saved_models/ffnn_2023-10-08T14:47:07.901649'

import yaml
with open('../gesture_data/gesture_info.yaml', 'r') as f:
    gesture_info = yaml.safe_load(f)['gestures']

sensors = list(common.read_constants('./constants.yaml')["sensors"].values())
df = pd.read_csv(
    csv_path,
    names=["datetime", "gesture"] + sensors,
    parse_dates=["datetime"],
    date_format='ISO8601',
)
df['file'] = csv_path

X, y, dt = common.make_windows(
    df,
    20,
    constants_path='../src/constants.yaml',
    pbar=tqdm.tqdm(total=len(df), desc="Making windows"),
)
clf = models.load_tf(model_dir)
y_pred = clf.predict(X)
y_pred_probs = clf.predict_proba(X)

## Plot data, labels, predictions over time

In [ ]:

start = 9000
duration = 420

def key(gidx): 
    gidx = gidx if gidx != 50 else 255
    return (
        gesture_info[f"gesture{gidx:0>4}"]["key"]
        .replace('space', "` '")
    )

def make_lbl(gidx): return f'{key(gidx)}\n{gidx}'

def pred_plot(start, finsh, y_pred_probs, X, y, dt):
    duration = finsh - start
    fig, axs = plt.subplots(2, 1, figsize=(WIDTH, 2.5), dpi=200)

    g50 = y_pred_probs[start:start+duration, 50]
    for idx in np.nonzero(g50 < 0.9)[0]:
        likely_preds = np.nonzero(y_pred_probs[start + idx] > 0.1)[0]
        probs_preds = [
            e for e in 
            sorted(
                zip(y_pred_probs[start + idx, likely_preds], likely_preds),
                key=lambda tpl: -tpl[0]
            )
            if e[1] != 50
        ]
        if not probs_preds:
            continue
        lbl = '\n'.join([
            f'{key(pred)}: {round(prob*100)}\%'
            for prob, pred in 
            probs_preds
        ])
#         print(
#             start + idx,
#             y[start + idx],
#             probs_preds
#         )
        true_val = y[start + idx]
        true_gidx = 50 if true_val == 'nan' else int(y[start + idx][-4:])
        pred_gidx = probs_preds[0][1]
        axs[0].text(
            idx, probs_preds[0][0], lbl,
            ha='center', va='bottom',
            bbox=dict(
                boxstyle="round",
                ec=(
                    'tab:green' 
                    if true_gidx == pred_gidx and probs_preds[0][0] > 0.5 else 
                    'tab:red'
                ),
                fc='#ddda',
            )
        )
        axs[0].plot(
            [idx, idx],
            [0, 1],
            c='k', alpha=0.5, linewidth=1, linestyle='dotted'
        )

    # For each class
    for gidx in range(y_pred_probs.shape[-1]):
        axs[0].plot(
            y_pred_probs[start:start+duration, gidx],
            alpha=0.5,
        )

    # print('plotting lines', flush=True)
    for i in range(X.shape[-1]):
        axs[1].plot(
            X[start:start+duration, -1, i],
            alpha=0.5,
            c=[mpl.colormaps['tab10'](i) for i in range(10)][i//3],
            lw=1,
        )
    axs[0].grid('both',lw=0.5, alpha=0.5)
    axs[1].grid('both',lw=0.5, alpha=0.5)

    for i, lbl in enumerate(y[start:start+duration]):
        if pd.isna(lbl) or lbl == 'nan' or lbl == 'gesture0255': continue
        gidx = int(lbl[-4:])
        axs[1].text(
            i, 800,
            key(gidx) if duration > 3000 else f'{key(gidx)}\n({gidx})',
            ha='center', va='bottom',
            bbox=None if duration > 3000 else dict(
                boxstyle="round",
                ec='#dddf',
                fc='#ddda',
            )
        )
        axs[1].plot(
            [i, i], [250, 950],
            c='k', alpha=0.5, linewidth=1, linestyle='dotted'
        )

    axs[0].set(
        ylabel='Probability',
        yticks=[0.0, 0.25, 0.5, 0.75, 1.0],
        xticks=[],
    )
    axs[1].set(
        ylabel='Sensor Value',
        xticks=[],
#         xticks=np.linspace(0, duration, 20).astype(int),
#         xticklabels=np.linspace(0, duration, 20).astype(int) + start,
        ylim=(250, 950),
        yticks=np.arange(300, 901, 100),
    )
    return fig, axs
    
fig, axs = pred_plot(
    900, 1800, y_pred_probs, X, y, dt
)
plt.show()

In [ ]:
start_finshs = [
#     (0, 9420, 'full_text'),
    (140, 750  , 'the'),
    (900, 1800 , 'quick'),
    (2050, 3000, 'brown'),
    (3460, 3900, 'fox'),
    (4000, 5800, 'jumped'),
    (5830, 6500, 'over'),
    (6700, 7270, 'the'),
    (7400, 8500, 'lazy'),
    (8560, 9150, 'dog'),
]

for start, finsh, text in start_finshs:
    print(start, finsh)
    fig, axs = pred_plot(
        start, finsh, y_pred_probs, X, y, dt
    )
    plt.tight_layout()
    plt.savefig(
        f'../../report/src/imgs/graphs/05_pred_plot_{start:0>4}_to_{finsh:0>4}_{text}.pdf'
    )
    plt.show()


## Make a confusion matrix and precision-recall heatmap

In [ ]:
%%script false --no-raise-error # This cell is unused

y_true = pd.Series(y).apply(lambda x: 50 if x in ('nan', 'gesture0255') else int(x[-4:]))
cm = tf.math.confusion_matrix(y_true, y_pred).numpy()
report = sklearn.metrics.classification_report(
    y_true,
    y_pred,
    output_dict=True,
    zero_division=0,
    labels=list(range(51))
)
fig, ax = plt.subplots(1, 1, figsize=(0.5*WIDTH, 0.5*WIDTH))
vis.conf_mat(cm, ax=ax)
ax.set(
    title='Confusion Matrix\nEnglish-language phrase'
)
plt.savefig(
    '../../report/src/imgs/graphs/05_eng_lang_phrase_conf_mat.pdf',
    bbox_inches='tight',
)
plt.show()
fig, ax = plt.subplots(1, 1, figsize=(0.5*WIDTH, 0.15*WIDTH))
vis.precision_recall_f1(report=report, ax=ax)
plt.savefig(
    '../../report/src/imgs/graphs/c.pdf',
    bbox_inches='tight',
)
plt.show

## Plot data, predictions, keystrokes over time

In [ ]:
%%script false --no-raise-error # This cell is unused
# clf=None
@interact(start=(0, len(df), 200), duration=(0, len(df), 200))
def fn(start=10, duration=500):
    fig, axs = plt.subplots(3 if clf is not None else 2, 1, figsize=(WIDTH, WIDTH))
    for i in range(X.shape[-1]):
        axs[0].plot(
            X[start:start+duration, 0, i],
            alpha=0.5,
            c=['tab:red', 'tab:green', 'tab:blue'][i%3],
            lw=1,
        )
    sns.heatmap(
        X[start:start+duration, 0, :].T,
        cmap='Spectral',
        ax=axs[1],
        cbar=False,
        vmin=290,
        vmax=910,
        yticklabels=5,
    )

    if clf is not None:
        sns.heatmap(
            y_pred_probs[start-10:start+duration-10, :].T,
            vmin=0,
            vmax=1,
            ax=axs[2],
            cbar=False,
            yticklabels=5,
            cmap='Spectral',
        )
        axs[2].set(
            ylabel='Predicted\ngesture',
        )
        for idx in np.nonzero(y_pred[start-10:start+duration-10] != 50)[0]:
            gidx = y_pred[start-10:start+duration-10][idx]
            gidx = 255 if gidx == 50 else gidx
            txt = gesture_info[f'gesture{gidx:0>4}']['key']
            axs[0].text(
                idx, 
                800,
                txt,
            )

    plt.subplots_adjust(hspace=0.05)
    
    axs[0].set(
        ylabel='Sensor value',
        xticks=[],
        ylim=(250, 950)
    )
    axs[1].set(
        ylabel='Sensor number',
        xticks=[],
    )
    axs[0].margins(0)
    with_clf =' and model predictions' if clf is not None else ''
    axs[0].set_title(
        f'Sensor values{with_clf} over time from {np.round(start/40, 2)}s\n'
        f'(duration: {np.round(duration/40, 2)} seconds)'
    )


## Easily convert text to gestures

In [ ]:
def char_to_gesture(c):
    c_old = c
    if c == ' ':
        c = 'space'
    for k, v in gesture_info.items():
        if v['key'] == c.lower():
            desc = (
                v["description"]
                .replace("l", "left ")
                .replace("r", "right")
                .replace(" 1", " thumb")
                .replace(" 2", " index")
                .replace(" 3", " middle")
                .replace(" 4", " ring")
                .replace(" 5", " pinky")
            )
            return f'[{c_old}] {k}  {desc}'
text = 'the quick brown fox jumps over the lazy dog'

print('\n'.join(char_to_gesture(c) for c in text))

# Misc Methodology chapter plots

In [ ]:
y_4_true = np.array([
    0, 1, 2, 3, 4, 5,
    0, 1, 2, 3, 4, 5,
    0, 1, 2, 3, 4, 5,
    0, 1, 2, 3, 4, 5,
    0, 1, 2, 3, 4, 5,
])
y_4_pred = np.array([
    0, 1, 2, 3, 4, 5,
    0, 1, 2, 3, 5, 4,
    0, 1, 2, 3, 4, 5,
    2, 2, 2, 2, 2, 2,
    0, 1, 2, 3, 4, 5,

])
conf_mat = tf.math.confusion_matrix(y_4_true, y_4_pred).numpy()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(WIDTH, WIDTH))

titles = np.array([
    ["Count of observations", "Normalized by columns"],
    ["Normalized by rows", "Normalized by total sum"],
])

for i in (0, 1):
    for j in (0, 1):
        if i == j == 0:
            div = 1
        elif i == j == 1:
            div = conf_mat.sum()
        elif i == 0 and j == 1:
            div = conf_mat.sum(axis=0)
        elif i == 1 and j == 0:
            div = conf_mat.sum(axis=1)
        sns.heatmap(
            conf_mat / div,
            square=True,
            annot=True,
            fmt='.0f' if i == j == 0 else '.2f',
            mask=conf_mat == 0,
            cmap='Spectral',
            vmin=0,
            vmax=conf_mat.max() if i == j == 0 else 1.0,
            ax=axs[i, j],
        )
        axs[i, j].set(
            title=titles[i, j],
            xlabel='Predicted',
            ylabel='Ground Truth',
        )
plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/04_example_conf_mat.pdf',
    bbox_inches='tight',
)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(WIDTH, WIDTH*.25))
report = sklearn.metrics.classification_report(
    y_4_true,
    y_4_pred,
    zero_division=0,
    output_dict=True,
)
vis.precision_recall_f1(report, ax=ax, annot=True)
# ax.set_xticks([0, 1, 2, 3, 4, 5]);
# ax.set_xticklabels([0, 1, 2, 3, 4, 5]);
ax.set(
    title='Precision, Recall, and $F_1$-score',
    xlabel='Class',
)
plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/04_prec_rec_f1_example.pdf',
    bbox_inches='tight',
)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(0.5*WIDTH, 0.5*WIDTH))

recall_grid, precision_grid = np.meshgrid(
    np.linspace(0, 1, 100), 
    np.linspace(0, 1, 100)
)
f1_score = 2 * (precision_grid * recall_grid) / (precision_grid + recall_grid)

contours = ax.contour(
    recall_grid, 
    precision_grid,
    f1_score, 
    levels=np.arange(0.1, 0.96, 0.1), 
    cmap='Spectral'
#     colors='black',
#     alpha=0.5
)
ax.clabel(contours, inline=True, fmt='%.2f')
ax.set(
    title='Precision vs Recall',
    ylabel='Recall',
    xlabel='Precision',
    xticks=np.arange(0, 1.01, 0.1),
    yticks=np.arange(0, 1.01, 0.1),
)

plt.savefig(
    '../../report/src/imgs/graphs/04_precision_recall_f1.pdf',
    bbox_inches='tight',
)
plt.show()

In [ ]:
y_4_true = np.array([
    5, 5, 5, 5, 5, 5, 5, 0, 0, 0,
    5, 5, 5, 5, 5, 5, 5, 1, 1, 1,
    5, 5, 5, 5, 5, 5, 5, 2, 2, 2,
    5, 5, 5, 5, 5, 5, 5, 3, 3, 3,
    5, 5, 5, 5, 5, 5, 5, 4, 4, 4,
])
y_4_pred = np.array([
    5, 5, 5, 5, 5, 5, 0, 5, 0, 0,
    5, 5, 5, 5, 5, 5, 1, 5, 5, 1,
    5, 5, 5, 5, 5, 5, 2, 5, 2, 2,
    5, 5, 5, 5, 5, 5, 3, 5, 4, 3,
    5, 5, 5, 5, 5, 5, 4, 5, 4, 4,

])
conf_mat = tf.math.confusion_matrix(y_4_true, y_4_pred).numpy()

sns.heatmap(
    conf_mat / conf_mat.sum(axis=1),
    square=True,
    annot=True,
#     fmt='.0f' if i == j == 0 else '.2f',
    mask=conf_mat == 0,
    cmap='Spectral',
)
plt.show()
report = sklearn.metrics.classification_report(
    y_4_true,
    y_4_pred,
    zero_division=0,
    output_dict=True,
)
vis.precision_recall_f1(report, annot=True)


# Misc Results Chapter Plots

### Read in all the data

In [ ]:
df = read.read_data(
    '../gesture_data/train/', 
    constants_path='../src/constants.yaml'
)
df['gidx'] = df['gesture'].apply(lambda g: int(g[-4:]) if g != 'gesture0255' else 50)
const = common.read_constants('../src/constants.yaml')
sensor_names = list(const['sensors'].values())

In [ ]:
df.columns

In [ ]:
df['datetime'].iloc[np.arange(s, f, 10)]

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(WIDTH, WIDTH))
s = 3270
f = 3350
idxs = np.arange(s, f)

axs[0].plot(
    df[sensor_names].iloc[idxs]
)
axs[0].set(
    xticks=np.arange(0, f - s, 10),
    yticks=np.arange(250, 950, 100),
    ylabel='Sensor Value',
    title='Sensor Values over Time',
)
axs[0].margins(x=0)
vis.add_grid(axs[0])

axs[1].imshow(
    df[sensor_names].iloc[idxs].T,
    cmap='Spectral'
)
# xticklabels = df['datetime'].iloc[np.arange(s, f, 10)].apply(lambda x: str(x)[:-3])
axs[1].set(
    ylabel='Sensor Number',
    yticks=np.arange(0, 30, 5),
    xlabel='Timestep',
    xticks=np.arange(0, f - s, 10),
    xticklabels=np.arange(s, f, 10),
)
plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/04_sensors_over_time.pdf',
    bbox_inches='tight',
)
plt.show()

## All observations of gesture XXX

In [ ]:
%%script false --no-raise-error # This cell is unused

def prettify_sensor_name(sensor_name):
    return {
        "l5x": "Left Little X",
        "l5y": "Left Little Y",
        "l5z": "Left Little Z",
        "l4x": "Left Ring X",
        "l4y": "Left Ring Y",
        "l4z": "Left Ring Z",
        "l3x": "Left Middle X",
        "l3y": "Left Middle Y",
        "l3z": "Left Middle Z",
        "l2x": "Left Index X",
        "l2y": "Left Index Y",
        "l2z": "Left Index Z",
        "l1x": "Left Thumb X",
        "l1y": "Left Thumb Y",
        "l1z": "Left Thumb Z",
        "r1x": "Right Thumb X",
        "r1y": "Right Thumb Y",
        "r1z": "Right Thumb Z",
        "r2x": "Right Index X",
        "r2y": "Right Index Y",
        "r2z": "Right Index Z",
        "r3x": "Right Middle X",
        "r3y": "Right Middle Y",
        "r3z": "Right Middle Z",
        "r4x": "Right Ring X",
        "r4y": "Right Ring Y",
        "r4z": "Right Ring Z",
        "r5x": "Right Little X",
        "r5y": "Right Little Y",
        "r5z": "Right Little Z",
    }.get(sensor_name, sensor_name)

for gidx in (0, 5, 11, 16, 22, 27, 33, 38, 44, 49):
    idxs = np.nonzero(df['gidx'] == gidx)[0][:20]
    cross_idxs = idxs[:, np.newaxis] + np.arange(-10, 21)
    fig, axs = plt.subplots(5, 6, figsize=(BIG_WIDTH, BIG_WIDTH*5/6))
    axs = vis.cmp_ts(
        df[sensor_names].values[cross_idxs],
        axs=axs,
        lw=0.25,
    )

    for i, ax in enumerate(axs.flatten()):
        ax.set_title(
            prettify_sensor_name(ax.get_title()),
            loc='center',
            y=0.95,
        )
        ax.set_ylim()
        ax.set(
            ylim=(250, 990),
            yticks=[300, 500, 700, 900],
            yticklabels=[] if i%6 != 0 else [300, 500, 700, 900],
            xticks=[5, 15, 25],
            xticklabels=[] if i+6 < len(axs.flat) else ['-10', '0', '+10'],
        )
        ax.grid('y', alpha=0.25, lw=1)
        ax.set_axisbelow(True)

#         vis.add_grid(ax)

    plt.suptitle(f'Gesture {gidx}')
    plt.tight_layout()
    plt.subplots_adjust(
        wspace=0.1,
        hspace=0.3,
    )
    plt.savefig(
        f'../../report/src/imgs/graphs/05_example_g{gidx:0>4}_plot.pdf',
        bbox_inches='tight',
    )
    plt.show()


## Correlations between the different gestures

In [ ]:
%%script false --no-raise-error # This cell is unused
timestep = 0
# Load in the dataset/classifier
(
    X_trn, X_val, y_trn, y_val, dt_trn, dt_val
) = common.read_and_split_from_npz("../gesture_data/trn_20_10.npz")
const = common.read_constants('../src/constants.yaml')
sensor_names = list(const['sensors'].values())
X_data = X_trn[y_trn != 50][:, timestep, :]
y_data = y_trn[y_trn != 50]

In [ ]:
%%script false --no-raise-error
fig, axs = plt.subplots(
    5, 10,
    figsize=(WIDTH, WIDTH*2),
    dpi=200,
)
for i in range(5):
    print(f'gesture {i}_', flush=True)
    for j in range(10):
        sns.heatmap(
            pd.DataFrame(X_data[y_data == (i * 10 + j)]).corr(),
            vmin=-1, 
            vmax=1,
            center=0,
            cbar=False,
            ax=axs[i, j],
            xticklabels=[s.upper() for s in sensor_names],
            yticklabels=[s.upper() for s in sensor_names],
            square=True,
        )
        axs[i, j].set_title(f'Gesture {i * 10 + j}')
plt.subplots_adjust(hspace=0.35, wspace=0.25)

In [ ]:
%%script false --no-raise-error # This cell is unused

sns.heatmap(
    pd.DataFrame(X_data).corr(),
    vmin=-1, 
    vmax=1,
    center=0,
#     cbar=False,
    xticklabels=[s.upper() for s in sensor_names],
    yticklabels=[s.upper() for s in sensor_names],
    square=True,
)
plt.xlabel('Sensor')
plt.ylabel('Sensor')
plt.title('Correlations between sensors\n(over all training data)')
plt.savefig(
    '../../report/src/imgs/graphs/05_correlations.pdf',
    bbox_inches='tight'
)

In [ ]:
%%script false --no-raise-error # This cell is unused

fig, axs = plt.subplots(1, 3, figsize=(WIDTH, WIDTH/3))
axis = ['X', 'Y', 'Z']
for i in range(3):
    sns.heatmap(
        pd.DataFrame(X_data[:, i::3]).corr(),
        vmin=-1, 
        vmax=1,
        center=0,
        cbar=False,
        square=True,
        ax=axs[i]
    )
    axs[i].set_title(f'Correlations between {axis[i]}-axis sensors\n(over all training data)')
plt.tight_layout()
plt.show()


## Time-series heatmap + line plots

In [ ]:
%%script false --no-raise-error # This cell is unused

def plt_subset(s, f):
    df = read.read_data(
        '../gesture_data/train/', 
        constants_path='../src/constants.yaml'
    )
    df['gidx'] = df['gesture'].apply(lambda g: int(g[-4:]) if g != 'gesture0255' else 50)
    const = common.read_constants('../src/constants.yaml')
    sensor_names = list(const['sensors'].values())
    data = df[sensor_names].values[s:f]
    
    fig, axs = plt.subplots(2, 1, figsize=(WIDTH, WIDTH*.5))
    sns.heatmap(
        data.T,
        ax=axs[0],
        cbar=False
    )

    axs[1].plot(
        data,
        color='black',
        alpha=0.2,
        linewidth=1,
    )
    plt.margins(0)
    plt.show()
# plt_subset(91_000, 95_000)
plt_subset(93_000, 93_200)

## Histogram of class distributions

In [ ]:
%%script false --no-raise-error # This cell is unused

df['gidx'].hist()
plt.show()
df.loc[df['gidx'] != 50, 'gidx'].hist()
df['gidx'].value_counts() / len(df['gidx']) * 100

## All observations of one gesture

In [ ]:
%%script false --no-raise-error # This cell is unused

gidx = 0
before = 10
after = 10
idxs = np.nonzero(df['gidx'] == gidx)[0][:, np.newaxis] + np.arange(-before, after+1)

vis.cmp_ts(df[sensor_names].values[idxs + 10]);
plt.tight_layout()
plt.show()